**The goal of the project is to develop a behavioral analytics system for identifying users based on unique mobile device movement patterns.**

# Content

* [Libraries and data import](#Libraries)
* [Functions](#Functions)
    * [EDA](#EDA-Functions)
    * [Feature Engineering](#Feature-Engineering-Functions)
    * [Modeling](#Modeling-Functions)
* [Exploratory data analysis](#EDA)
* [Feature Engineering](#Feature-Engineering)
* [Cluster Analysis](#Cluster-Analysis)
* [Modeling](#Modeling)
    * [Models Training](#Models-Training)
    * [Models Results](#Models-Results)
    * [Models + PCA](#Model-PCA)
    * [Models Different Features combinations](#Model-FS)
    * [Best Model](#Best-Model)
* [Final Prediction](#Conclusion)

# Libraries and data import <a name="Libraries"></a>

In [ ]:

import time
import warnings
import itertools
from collections import defaultdict

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Statistics
from scipy import stats
from scipy.stats import normaltest, shapiro, kstest, anderson

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.stattools import acf, pacf, adfuller

#Models
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    VotingClassifier,
    StackingClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    StratifiedKFold,
    GridSearchCV,
    RandomizedSearchCV,
)
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_auc_score,
    roc_curve,
    auc,
    matthews_corrcoef,
    cohen_kappa_score,
    silhouette_score,
    adjusted_rand_score,
)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, regularizers

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform


In [ ]:
df = pd.read_csv('/content/train.csv')
df.head()

,uid,timestamp,sensor_type,field_0,field_1,field_2,field_3,field_4,field_5,field_6,field_7,session_id,user_id
0,0157a1be-e373-4ca3-bc1f-f43885d11321,1758385959602,1,0.140499,0.403171,-0.185703,NaN,NaN,NaN,NaN,NaN,6c92cd0b-027c-4a0c-9802-4af636918681,e2ec300d-bbd6-4d29-a7c8-505970601891
1,04911357-2f63-4577-8c3b-0cd441319368,1758385960618,2,1.628057,4.774038,8.216900,NaN,NaN,NaN,NaN,NaN,6c92cd0b-027c-4a0c-9802-4af636918681,e2ec300d-bbd6-4d29-a7c8-505970601891
2,0554c28b-4992-4009-80e5-d4ce68236a03,1758385933842,2,0.737414,4.955997,9.150639,NaN,NaN,NaN,NaN,NaN,6c92cd0b-027c-4a0c-9802-4af636918681,e2ec300d-bbd6-4d29-a7c8-505970601891
3,0ad3f5f4-7266-4404-9f3a-3daae0486477,1758385977802,2,0.723049,6.138733,7.661446,NaN,NaN,NaN,NaN,NaN,6c92cd0b-027c-4a0c-9802-4af636918681,e2ec300d-bbd6-4d29-a7c8-505970601891
4,11137f53-7b69-4d79-a93e-8c87dae8f536,1758385922522,2,0.407014,5.559336,7.752425,NaN,NaN,NaN,NaN,NaN,6c92cd0b-027c-4a0c-9802-4af636918681,e2ec300d-bbd6-4d29-a7c8-505970601891


In [ ]:
df_test = pd.read_csv('/content/test.csv')
df_test.head()

,uid,timestamp,sensor_type,field_0,field_1,field_2,field_3,field_4,field_5,field_6,field_7,session_id
0,00fb2624-996e-4dde-a9b9-5a0d6784abfa,1755089347628,1,0.023366,0.043524,0.067501,NaN,NaN,NaN,NaN,NaN,a47ea05f-68b6-4e84-8591-db9580d58ce6
1,03b1d813-f3e1-481f-b523-8470ba81fb96,1755089326732,6,0.388720,0.403039,0.658245,0.503161,-1.0,-1.183342,-0.263185,-2.053627,a47ea05f-68b6-4e84-8591-db9580d58ce6
2,0995b64c-693c-4fc1-ac12-5d3e5e4d253c,1755089328671,4,0.751356,9.419654,2.622206,NaN,NaN,NaN,NaN,NaN,a47ea05f-68b6-4e84-8591-db9580d58ce6
3,0a852d2e-c12c-4b71-9f79-d801ce07c9f2,1755089369479,2,-0.311067,9.467801,2.925226,NaN,NaN,NaN,NaN,NaN,a47ea05f-68b6-4e84-8591-db9580d58ce6
4,1df4588a-5f0f-46e6-bcd2-38e750eec0a7,1755089230747,2,1.351945,9.412168,2.305485,NaN,NaN,NaN,NaN,NaN,a47ea05f-68b6-4e84-8591-db9580d58ce6


# Functions <a name="Functions"></a>

## EDA <a name="Eda-Functions"></a>



In [ ]:

# sensor mapping
SENSOR_TYPES = {
    1: 'Gyroscope',
    2: 'Accelerometer',
    4: 'Gravity',
    5: 'Magnetometer',
    6: 'Rotation',
    19: 'UserAcceleration'
}


def load_and_clean_data(df : pd.DataFrame) -> pd.DataFrame:
    """
    Clean df
    """
    df_clean = df.copy()

    # Replace empty strings and string 'nan' with actual NaN values
    df_clean['session_id'] = df_clean['session_id'].replace({'': np.nan, 'nan': np.nan})
    df_clean['user_id'] = df_clean['user_id'].replace({'': np.nan, 'nan': np.nan})

    # Drop rows
    initial_rows = len(df_clean)
    df_clean.dropna(subset=['session_id', 'user_id'], inplace=True)
    cleaned_rows = len(df_clean)

    if initial_rows - cleaned_rows > 0:
        print(f"Dropped {initial_rows - cleaned_rows} rows due to missing session_id or user_id.")

    print(f"Data loaded: {df_clean.shape[0]} rows, {df_clean.shape[1]} columns")

    return df_clean


def df_info(df : pd.DataFrame)-> None:
    """
    data info
    """
    print("\n" + "="*80)
    print("DF Information")
    print("="*80)

    print("\nDataset Shape:", df.shape)
    print("\nColumn Data Types:")
    print(df.dtypes)

    print("\nMissing Values:")
    missing = df.isnull().sum()
    missing_pct = 100 * missing / len(df)
    missing_df = pd.DataFrame({
        'Missing_Count': missing,
        'Percentage': missing_pct
    }).sort_values('Percentage', ascending=False)
    print(missing_df[missing_df['Missing_Count'] > 0])

    print("\nBasic Statistics:")
    print(df.describe())

    print("\nSensor Type Distribution:")
    print(df['sensor_type'].value_counts().sort_index())

    print("\nNumber of Unique Users:", df['user_id'].nunique())
    print("Number of Unique Sessions:", df['session_id'].nunique())

    sessions_per_user = df.groupby('user_id')['session_id'].nunique()
    print(f"\nSessions per User - Mean: {sessions_per_user.mean():.2f}, "
          f"Median: {sessions_per_user.median():.2f}, "
          f"Min: {sessions_per_user.min()}, Max: {sessions_per_user.max()}")

    events_per_session = df.groupby('session_id').size()
    print(f"\nEvents per Session - Mean: {events_per_session.mean():.2f}, "
          f"Median: {events_per_session.median():.2f}, "
          f"Min: {events_per_session.min()}, Max: {events_per_session.max()}")


def visualize_data(df : pd.DataFrame)-> None:
    """
    visualisation of data

    """

    #Sensor Type Distribution
    sensor_counts = df['sensor_type'].map(SENSOR_TYPES).value_counts()
    fig1 = px.bar(x=sensor_counts.index, y=sensor_counts.values,
                  labels={'x': 'Sensor Type', 'y': 'Number of Events'},
                  title='Distribution of Sensor Events',
                  color=sensor_counts.values,
                  color_continuous_scale='viridis')
    fig1.show()

    #Events per User
    events_per_user = df.groupby('user_id').size().sort_values(ascending=False)
    fig2 = px.bar(y=events_per_user.values,
                  labels={'y': 'Number of Events', 'index': 'User Index'},
                  title='Events per User Distribution',
                  color=events_per_user.values,
                  color_continuous_scale='plasma')
    fig2.show()

    #Sessions per User
    sessions_per_user = df.groupby('user_id')['session_id'].nunique().sort_values(ascending=False)
    fig3 = px.bar(y=sessions_per_user.values,
                  labels={'y': 'Number of Sessions', 'index': 'User Index'},
                  title='Sessions per User Distribution',
                  color=sessions_per_user.values,
                  color_continuous_scale='turbo')
    fig3.show()

    # Temporal distribution
    df_temp = df.copy()
    df_temp['datetime'] = pd.to_datetime(df_temp['timestamp'], unit='ms')
    hourly_dist = df_temp['datetime'].dt.hour.value_counts().sort_index()
    fig4 = px.line(x=hourly_dist.index, y=hourly_dist.values,
                   labels={'x': 'Hour of Day', 'y': 'Number of Events'},
                   title='Temporal Distribution of Events (by Hour)',
                   markers=True)
    fig4.show()

    numerical_cols = df.select_dtypes(include=np.number).columns
    correlation_matrix = df[numerical_cols].corr()
    fig5 = px.imshow(correlation_matrix,
                text_auto=True,
                aspect="auto",
                title="Correlation Matrix of Numerical Features")
    fig5.update_layout(xaxis_title="Numerical Features", yaxis_title="Numerical Features")
    fig5.show()



def analyze_sensor_signals(df : pd.DataFrame, sample_size=5000) -> None:
    """
    Statistical analysis for sensor signals.
    """

    # Sample data for visualization
    sample_df = df.sample(min(sample_size, len(df)), random_state=42)

    for sensor_id, sensor_name in SENSOR_TYPES.items():
        sensor_data = df[df['sensor_type'] == sensor_id]

        if len(sensor_data) == 0:
            continue

        print(f"\n{sensor_name} (Type {sensor_id}):")
        print(f"  Total events: {len(sensor_data)}")

        # Determine fields to analyze sinse Rotation has 7 fields
        fields_to_analyze = []
        if sensor_id in [1, 2, 4, 5, 19]:  # 3-axis sensors
            fields_to_analyze = ['field_0', 'field_1', 'field_2']
        elif sensor_id == 6:  # Rotation sensor
            fields_to_analyze = ['field_0', 'field_1', 'field_2', 'field_3',
                                'field_5', 'field_6', 'field_7']

        for field in fields_to_analyze:
            values = sensor_data[field].dropna()
            if len(values) < 10:
                continue

            # Basic statistics
            print(f"\n  {field}:")
            print(f"    Mean: {values.mean():.4f}, Std: {values.std():.4f}")
            print(f"    Min: {values.min():.4f}, Max: {values.max():.4f}")
            print(f"    Skewness: {values.skew():.4f}, Kurtosis: {values.kurtosis():.4f}")

            # Normality tests
            if len(values) >= 20:
                try:
                    # Shapiro-Wilk test
                    if len(values) < 5000:
                        stat, p_value = shapiro(values.sample(min(1000, len(values))))
                        print(f"    Shapiro-Wilk test: p-value = {p_value:.4e}")

                    # D'Agostino's K-squared test
                    stat, p_value = normaltest(values.sample(min(1000, len(values))))
                    print(f"    Normality test (D'Agostino): p-value = {p_value:.4e}")
                except Exception as e:
                    print(f"    Normality test failed: {str(e)}")

    # Visualize signals
    _visualize_sensor_signals(sample_df)


def _visualize_sensor_signals(sample_df):
    """
    Visualize sensor signals
    """

    # Accelerometer signals
    accel_data = sample_df[sample_df['sensor_type'] == 2].sort_values('timestamp').head(500)
    if len(accel_data) > 0:
        fig = go.Figure()
        fig.add_trace(go.Scatter(y=accel_data['field_0'], mode='lines', name='X-axis'))
        fig.add_trace(go.Scatter(y=accel_data['field_1'], mode='lines', name='Y-axis'))
        fig.add_trace(go.Scatter(y=accel_data['field_2'], mode='lines', name='Z-axis'))
        fig.update_layout(title='Accelerometer Signals (Sample)',
                        xaxis_title='Event Index',
                        yaxis_title='Acceleration (m/s²)',
                        hovermode='x unified')
        fig.show()

    # Gyroscope signals
    gyro_data = sample_df[sample_df['sensor_type'] == 1].sort_values('timestamp').head(500)
    if len(gyro_data) > 0:
        fig = go.Figure()
        fig.add_trace(go.Scatter(y=gyro_data['field_0'], mode='lines', name='X-axis'))
        fig.add_trace(go.Scatter(y=gyro_data['field_1'], mode='lines', name='Y-axis'))
        fig.add_trace(go.Scatter(y=gyro_data['field_2'], mode='lines', name='Z-axis'))
        fig.update_layout(title='Gyroscope Signals (Sample)',
                        xaxis_title='Event Index',
                        yaxis_title='Angular Velocity (rad/s)',
                        hovermode='x unified')
        fig.show()

#Autocorrelation
def test_autocorrelation(df : pd.DataFrame, max_lags=50) -> None:
    """
    Test for autocorrelation in sensor signals.
    """

    # Sample one session for detailed analysis
    sample_session = df['session_id'].iloc[0]
    session_data = df[df['session_id'] == sample_session].sort_values('timestamp')

    print(f"\nAnalyzing autocorrelation for session: {sample_session}")

    # Store ACF results for all sensors
    acf_results = {}
    _visualize_autocorrelation(acf_results, max_lags)

    # Analyze each sensor type
    for sensor_id, sensor_name in SENSOR_TYPES.items():
        sensor_data = session_data[session_data['sensor_type'] == sensor_id]

        if len(sensor_data) < max_lags + 10:
            print(f"\n{sensor_name}: Insufficient data (only {len(sensor_data)} events)")
            continue

        print(f"\n{sensor_name} (Type {sensor_id}):")
        print(f"  Total events: {len(sensor_data)}")

        acf_results[sensor_name] = {}

        # Determine which fields to analyze based on sensor type
        if sensor_id in [1, 2, 4, 5, 19]:  # 3-axis sensors
            fields_to_analyze = ['field_0', 'field_1', 'field_2']
            field_names = ['X-axis', 'Y-axis', 'Z-axis']
        elif sensor_id == 6:  # Rotation sensor
            fields_to_analyze = ['field_0', 'field_1', 'field_2', 'field_5', 'field_6', 'field_7']
            field_names = ['X-rotation', 'Y-rotation', 'Z-rotation', 'Pitch', 'Roll', 'Yaw']
        else:
            continue

        for field, field_name in zip(fields_to_analyze, field_names):
            values = sensor_data[field].dropna().values

            if len(values) < max_lags + 10:
                continue

            # Calculate ACF
            acf_values = acf(values, nlags=max_lags, fft=True)
            acf_results[sensor_name][field_name] = acf_values

            # Ljung-Box test for autocorrelation
            try:
                lb_test = acorr_ljungbox(values, lags=[10, 20], return_df=True)

                print(f"\n  {field_name} ({field}):")
                print(f"    Ljung-Box test:")
                print(f"      Lag 10: p-value = {lb_test['lb_pvalue'].iloc[0]:.4e}")
                print(f"      Lag 20: p-value = {lb_test['lb_pvalue'].iloc[1]:.4e}")

                # ADF test for stationarity
                adf_result = adfuller(values)
                print(f"    Augmented Dickey-Fuller test:")
                print(f"      Test statistic: {adf_result[0]:.4f}")
                print(f"      p-value: {adf_result[1]:.4e}")
                print(f"      Stationary: {'Yes' if adf_result[1] < 0.05 else 'No'}")

                # Additional statistics
                print(f"    Signal characteristics:")
                print(f"      Mean: {np.mean(values):.4f}")
                print(f"      Std: {np.std(values):.4f}")
                print(f"      Autocorr at lag 1: {acf_values[1]:.4f}")
                print(f"      Autocorr at lag 10: {acf_values[10]:.4f}")

            except Exception as e:
                print(f"    Error in statistical tests: {str(e)}")


        # Visualize ACF
    _visualize_autocorrelation(acf_results, max_lags)


def _visualize_autocorrelation(acf_results : pd.DataFrame, max_lags : int):
    for sensor_name, fields_data in acf_results.items():
        if not fields_data:
            continue

        num_fields = len(fields_data)
        cols = min(3, num_fields)
        rows = (num_fields + cols - 1) // cols

        subplot_titles = list(fields_data.keys())

        fig = make_subplots(
            rows=rows,
            cols=cols,
            subplot_titles=subplot_titles
        )

        for idx, (field_name, acf_values) in enumerate(fields_data.items(), 1):
            row = (idx - 1) // cols + 1
            col = (idx - 1) % cols + 1

            fig.add_trace(
                go.Bar(
                    x=list(range(len(acf_values))),
                    y=acf_values,
                    name=field_name,
                    showlegend=False
                ),
                row=row, col=col
            )

            confidence = 1.96 / np.sqrt(len(acf_values) * 10)
            fig.add_hline(
                y=confidence,
                line_dash="dash",
                line_color="red",
                row=row, col=col
            )
            fig.add_hline(
                y=-confidence,
                line_dash="dash",
                line_color="red",
                row=row, col=col
            )

        fig.update_layout(
            title=f'Autocorrelation Function - {sensor_name}',
            height=300 * rows,
            showlegend=False
        )
        fig.update_xaxes(title_text="Lag")
        fig.update_yaxes(title_text="ACF")
        fig.show()



In [221]:
#Clutserisation functions
def analyze_user_clusters(features_df: pd.DataFrame, feature_names: list) -> dict:

    X = features_df[feature_names].values
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(features_df['user_id'])
    user_ids = features_df['user_id'].values

    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # t-SNE visualization
    print("\nPerforming t-SNE dimensionality reduction...")
    tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
    X_tsne = tsne.fit_transform(X_scaled)

    tsne_df = pd.DataFrame({
        'tsne_1': X_tsne[:, 0],
        'tsne_2': X_tsne[:, 1],
        'user_id': user_ids,
        'user_label': y
    })

    # 2D t-SNE plot
    fig = px.scatter(
        tsne_df, x='tsne_1', y='tsne_2',
        color='user_id',
        title='t-SNE Visualization of User Sessions (2D)',
        labels={'tsne_1': 't-SNE Component 1', 'tsne_2': 't-SNE Component 2'},
        opacity=0.7,
        hover_data=['user_id']
    )
    fig.update_layout(height=700)
    fig.show()

    # Hierarchical clustering

    # Calculate mean features per user
    user_profiles = features_df.groupby('user_id')[feature_names].mean()
    user_profiles_scaled = scaler.fit_transform(user_profiles)

    # Linkage
    linkage_matrix = linkage(user_profiles_scaled, method='ward')

    # Create dendrogram
    fig = go.Figure()

    # Calculate dendrogram
    dend = dendrogram(linkage_matrix, labels=user_profiles.index.tolist(),
                           no_plot=True)

    # Plot dendrogram
    icoord = np.array(dend['icoord'])
    dcoord = np.array(dend['dcoord'])
    colors = dend['color_list']

    # Map matplotlib colors to plotly-compatible colors
    color_map = {
        'C0': '#1f77b4', 'C1': '#ff7f0e', 'C2': '#2ca02c', 'C3': '#d62728',
        'C4': '#9467bd', 'C5': '#8c564b', 'C6': '#e377c2', 'C7': '#7f7f7f',
        'C8': '#bcbd22', 'C9': '#17becf', 'b': 'blue', 'g': 'green',
        'r': 'red', 'c': 'cyan', 'm': 'magenta', 'y': 'yellow', 'k': 'black'
    }

    for i in range(len(icoord)):
        # Convert matplotlib color to plotly color
        color = colors[i]
        plotly_color = color_map.get(color, 'gray')

        fig.add_trace(go.Scatter(
            x=icoord[i], y=dcoord[i],
            mode='lines',
            line=dict(color=plotly_color, width=2),
            showlegend=False,
            hoverinfo='skip'
        ))

    fig.update_layout(
        title='Hierarchical Clustering of Users (Ward Linkage)',
        xaxis_title='User ID',
        yaxis_title='Distance',
        height=600,
        xaxis=dict(tickvals=list(range(5, len(user_profiles)*10+5, 10)),
                  ticktext=dend['ivl'])
    )
    fig.show()

    # K-means clustering
    silhouette_scores = []
    ari_scores = []
    k_range = range(2, 11)

    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(X_scaled)

        sil_score = silhouette_score(X_scaled, cluster_labels)
        ari_score = adjusted_rand_score(y, cluster_labels)

        silhouette_scores.append(sil_score)
        ari_scores.append(ari_score)

    print("\nClustering Quality Metrics:")
    for k, sil, ari in zip(k_range, silhouette_scores, ari_scores):
        print(f"  k={k}: Silhouette={sil:.4f}, ARI={ari:.4f}")

    # Visualize clustering
    fig = make_subplots(rows=1, cols=2,
                       subplot_titles=('Silhouette Score vs K',
                                     'Adjusted Rand Index vs K'))

    fig.add_trace(
        go.Scatter(x=list(k_range), y=silhouette_scores,
                  mode='lines+markers', name='Silhouette',
                  line=dict(color='blue', width=3)),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=list(k_range), y=ari_scores,
                  mode='lines+markers', name='ARI',
                  line=dict(color='red', width=3)),
        row=1, col=2
    )

    fig.update_xaxes(title_text="Number of Clusters (k)", row=1, col=1)
    fig.update_xaxes(title_text="Number of Clusters (k)", row=1, col=2)
    fig.update_yaxes(title_text="Silhouette Score", row=1, col=1)
    fig.update_yaxes(title_text="Adjusted Rand Index", row=1, col=2)
    fig.update_layout(height=500, showlegend=False)
    fig.show()

    # Intra-class distance heatmap

    # Calculate pairwise distances
    distances = squareform(pdist(X_scaled, metric='euclidean'))

    # Sort by user_id for block structure
    sort_idx = np.argsort(y)
    distances_sorted = distances[sort_idx][:, sort_idx]
    y_sorted = y[sort_idx]

    max_samples = 150
    if len(distances_sorted) > max_samples:
        step = len(distances_sorted) // max_samples
        distances_sorted = distances_sorted[::step, ::step]
        y_sorted = y_sorted[::step]

    fig = go.Figure(data=go.Heatmap(
        z=distances_sorted,
        colorscale='Viridis',
        colorbar=dict(title='Euclidean Distance')
    ))

    fig.update_layout(
        title='Pairwise Session Distance Matrix (sorted by User ID)',
        xaxis_title='Session Index',
        yaxis_title='Session Index',
        height=700,
        width=700
    )
    fig.show()

    return {
        'tsne_df': tsne_df,
        'linkage_matrix': linkage_matrix,
        'silhouette_scores': silhouette_scores,
        'ari_scores': ari_scores,
        'X_scaled': X_scaled,
        'y': y
    }


def profile_extreme_users(features_df: pd.DataFrame, feature_names: list) -> dict:
    """
    Get special users for  analysis of behaviour.
    """

    # Calculate user statistics
    user_stats = features_df.groupby('user_id').agg({
        'event_count': 'mean',
        'session_duration': 'mean',
        'Accelerometer_magnitude_mean': 'mean',
        'Accelerometer_energy': 'mean',
        'Gyroscope_magnitude_std': 'mean',
        'avg_event_interval': 'mean'
    })

    # Identify extreme users
    most_active = user_stats['event_count'].idxmax()
    most_static = user_stats['Accelerometer_energy'].idxmin()
    most_dynamic = user_stats['Gyroscope_magnitude_std'].idxmax()

    print(f"\n Most Active User: {most_active}")
    print(f"   Average events per session: {user_stats.loc[most_active, 'event_count']:.0f}")
    print(f"   Average session duration: {user_stats.loc[most_active, 'session_duration']:.2f}s")

    print(f"\n Most Static User: {most_static}")
    print(f"   Average accelerometer energy: {user_stats.loc[most_static, 'Accelerometer_energy']:.2f}")
    print(f"   Average magnitude: {user_stats.loc[most_static, 'Accelerometer_magnitude_mean']:.4f}")

    print(f"\n Most Dynamic User: {most_dynamic}")
    print(f"   Gyroscope std: {user_stats.loc[most_dynamic, 'Gyroscope_magnitude_std']:.4f}")
    print(f"   Average event interval: {user_stats.loc[most_dynamic, 'avg_event_interval']:.2f}ms")

    # Store extreme users
    extreme_users = {
        'most_active': most_active,
        'most_static': most_static,
        'most_dynamic': most_dynamic
    }

    # Radar chart comparison
    _user_radar_chart(features_df, feature_names, [most_active, most_static, most_dynamic], extreme_users)

    # Detailed profile comparison
    _user_profile_comparison(features_df, feature_names, [most_active, most_static, most_dynamic], extreme_users)

    return extreme_users


def _user_radar_chart(features_df : pd.DataFrame, feature_names : list, user_ids : list, extreme_users : dict) -> None:

    # key features for radar chart
    key_features = [
        'event_count',
        'session_duration',
        'Accelerometer_magnitude_mean',
        'Accelerometer_energy',
        'Gyroscope_magnitude_std',
        'avg_event_interval',
        'Magnetometer_ratio',
        'Gravity_magnitude_max'
    ]

    # Calculate mean values per user
    user_profiles = features_df.groupby('user_id')[key_features].mean()

    # Normalize to 0-1 range
    scaler = MinMaxScaler()
    user_profiles_norm = pd.DataFrame(
        scaler.fit_transform(user_profiles),
        index=user_profiles.index,
        columns=user_profiles.columns
    )

    # Create radar chart
    fig = go.Figure()

    user_labels = {
        extreme_users['most_active']: 'Most Active',
        extreme_users['most_static']: 'Most Static',
        extreme_users['most_dynamic']: 'Most Dynamic'
    }

    colors = ['red', 'blue', 'green']

    for idx, user_id in enumerate(user_ids):
        values = user_profiles_norm.loc[user_id].values.tolist()
        values.append(values[0])

        fig.add_trace(go.Scatterpolar(
            r=values,
            theta=key_features + [key_features[0]],
            fill='toself',
            name=user_labels.get(user_id, user_id),
            line=dict(color=colors[idx], width=2)
        ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, range=[0, 1])
        ),
        showlegend=True,
        title='User Behavioral Profile Comparison (Normalized)',
        height=600
    )
    fig.show()


def _user_profile_comparison(features_df : pd.DataFrame, feature_names : list, user_ids : list, extreme_users :dict) -> None:

    # Parallel coordinates plot
    key_features = [
        'event_count', 'session_duration', 'avg_event_interval',
        'Accelerometer_magnitude_mean', 'Accelerometer_energy',
        'Gyroscope_magnitude_std', 'Gravity_magnitude_max',
        'Magnetometer_ratio'
    ]

    # Get all user profiles
    user_profiles = features_df.groupby('user_id')[key_features].mean()

    # Normalize
    scaler = MinMaxScaler()
    user_profiles_norm = pd.DataFrame(
        scaler.fit_transform(user_profiles),
        index=user_profiles.index,
        columns=user_profiles.columns
    )

    # Add user_id column
    user_profiles_norm['user_id'] = user_profiles_norm.index

    # Create dimension list for parallel coordinates
    dimensions = []
    for feature in key_features:
        dimensions.append(dict(
            range=[0, 1],
            label=feature.replace('_', ' ').title(),
            values=user_profiles_norm[feature]
        ))

    # Highlight extreme users
    user_profiles_norm['color'] = 0
    for idx, user_id in enumerate(user_ids):
        user_profiles_norm.loc[user_id, 'color'] = idx + 1

    fig = go.Figure(data=
        go.Parcoords(
            line=dict(
                color=user_profiles_norm['color'],
                colorscale=[[0, 'lightgray'], [0.33, 'red'],
                           [0.66, 'blue'], [1, 'green']],
                showscale=True,
                cmin=0,
                cmax=3
            ),
            dimensions=dimensions
        )
    )

    fig.update_layout(
        title='Parallel Coordinates: All Users (Extreme Users Highlighted)',
        height=600
    )
    fig.show()


def analyze_user_consistency(features_df : pd.DataFrame, feature_names : list) -> pd.DataFrame:
    """
    Analyze within-user consistency across sessions.
    """

    # Select key features
    key_features = [
        'session_duration', 'event_count', 'avg_event_interval',
        'Accelerometer_magnitude_mean', 'Gyroscope_magnitude_std',
        'Gravity_magnitude_mean'
    ]

    # Calculate coefficient of variation per user
    user_cv = features_df.groupby('user_id')[key_features].apply(
        lambda x: x.std() / x.mean()
    )

    print("\nCoefficient of Variation by User:")
    print("Lower CV = Higher consistency")
    print("\nMean CV across features:")
    print(user_cv.mean(axis=1).sort_values())

    # Most consistent user
    most_consistent_user = user_cv.mean(axis=1).idxmin()
    least_consistent_user = user_cv.mean(axis=1).idxmax()

    print(f"\nMost Consistent User: {most_consistent_user} (CV={user_cv.mean(axis=1).min():.4f})")
    print(f"Least Consistent User: {least_consistent_user} (CV={user_cv.mean(axis=1).max():.4f})")

    # Boxplot comparison
    fig = go.Figure()

    for feature in key_features:
        # Sample 5 users for clarity
        sample_users = features_df['user_id'].unique()[:5]

        for user_id in sample_users:
            user_data = features_df[features_df['user_id'] == user_id][feature]

            fig.add_trace(go.Box(
                y=user_data,
                name=f"{user_id}",
                boxmean='sd',
                visible=(feature == key_features[0])
            ))

    # Create dropdown buttons
    buttons = []
    for idx, feature in enumerate(key_features):
        visible = [False] * (len(key_features) * 5)
        for i in range(5):
            visible[idx * 5 + i] = True

        buttons.append(dict(
            label=feature,
            method="update",
            args=[{"visible": visible},
                  {"title": f"Within-User Variability: {feature}"}]
        ))

    fig.update_layout(
        updatemenus=[dict(
            active=0,
            buttons=buttons,
            direction="down",
            showactive=True,
            x=0.17,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )],
        title=f"Within-User Variability: {key_features[0]}",
        yaxis_title="Feature Value",
        xaxis_title="User ID",
        height=600,
        showlegend=True
    )
    fig.show()

    # Heatmap of CV
    fig2 = go.Figure(data=go.Heatmap(
        z=user_cv.values,
        x=user_cv.columns,
        y=user_cv.index,
        colorscale='RdYlGn_r',
        colorbar=dict(title='Coefficient of Variation')
    ))

    fig2.update_layout(
        title='User Consistency Heatmap ',
        xaxis_title='Feature',
        yaxis_title='User ID',
        height=600
    )
    fig2.update_xaxes(tickangle=45)
    fig2.show()

    return user_cv

## Feature Engineering <a name="Feature-Engineering-Functions"></a>

In [260]:
def engineer_features(df : pd.DataFrame) -> pd.DataFrame:

    features_list = []

    for session_id in df['session_id'].unique():
        session_data = df[df['session_id'] == session_id]
        user_id = session_data['user_id'].iloc[0]

        features = {'session_id': session_id, 'user_id': user_id}

        # Temporal features
        timestamps = session_data['timestamp'].values
        features['session_duration'] = (timestamps.max() - timestamps.min()) / 1000  # seconds
        features['event_count'] = len(session_data)
        features['avg_event_interval'] = np.mean(np.diff(sorted(timestamps))) if len(timestamps) > 1 else 0
        features['std_event_interval'] = np.std(np.diff(sorted(timestamps))) if len(timestamps) > 1 else 0

        # Sensor distribution features
        sensor_counts = session_data['sensor_type'].value_counts()
        for sensor_id, sensor_name in SENSOR_TYPES.items():
            features[f'{sensor_name}_count'] = sensor_counts.get(sensor_id, 0)
            features[f'{sensor_name}_ratio'] = sensor_counts.get(sensor_id, 0) / len(session_data)

        # Statistical features for each sensor
        for sensor_id, sensor_name in SENSOR_TYPES.items():
            sensor_subset = session_data[session_data['sensor_type'] == sensor_id]

            if len(sensor_subset) == 0:
                # Fill with zeros if sensor not present
                for stat in ['mean', 'std', 'min', 'max', 'median', 'q25', 'q75', 'iqr', 'skew', 'kurt']:
                    for i in range(8):  # max fields
                        features[f'{sensor_name}_field{i}_{stat}'] = 0
                continue

            # Analyze fields
            for i in range(8):
                field_name = f'field_{i}'
                if field_name in sensor_subset.columns:
                    values = sensor_subset[field_name].dropna()

                    if len(values) > 0:
                        features[f'{sensor_name}_field{i}_mean'] = values.mean()
                        features[f'{sensor_name}_field{i}_std'] = values.std()
                        features[f'{sensor_name}_field{i}_min'] = values.min()
                        features[f'{sensor_name}_field{i}_max'] = values.max()
                        features[f'{sensor_name}_field{i}_median'] = values.median()
                        features[f'{sensor_name}_field{i}_q25'] = values.quantile(0.25)
                        features[f'{sensor_name}_field{i}_q75'] = values.quantile(0.75)
                        features[f'{sensor_name}_field{i}_iqr'] = values.quantile(0.75) - values.quantile(0.25)
                        features[f'{sensor_name}_field{i}_skew'] = values.skew() if len(values) > 2 else 0
                        features[f'{sensor_name}_field{i}_kurt'] = values.kurtosis() if len(values) > 3 else 0

                        # Magnitude and energy features for 3-axis sensors
                        if i < 3 and sensor_id in [1, 2, 4, 5, 19]:
                            if i == 0:  # Calculate once for all 3 axes
                                x_vals = sensor_subset['field_0'].dropna()
                                y_vals = sensor_subset['field_1'].dropna()
                                z_vals = sensor_subset['field_2'].dropna()

                                min_len = min(len(x_vals), len(y_vals), len(z_vals))
                                if min_len > 0:
                                    magnitude = np.sqrt(x_vals[:min_len]**2 +
                                                      y_vals[:min_len]**2 +
                                                      z_vals[:min_len]**2)
                                    features[f'{sensor_name}_magnitude_mean'] = magnitude.mean()
                                    features[f'{sensor_name}_magnitude_std'] = magnitude.std()
                                    features[f'{sensor_name}_magnitude_max'] = magnitude.max()

                                    # Energy
                                    energy = np.sum(magnitude**2)
                                    features[f'{sensor_name}_energy'] = energy
                    else:
                        # Fill with zeros
                        for stat in ['mean', 'std', 'min', 'max', 'median', 'q25', 'q75', 'iqr', 'skew', 'kurt']:
                            features[f'{sensor_name}_field{i}_{stat}'] = 0

        features_list.append(features)

    # Create DataFrame
    features_df = pd.DataFrame(features_list)

    # Fill any remaining NaN values
    features_df = features_df.fillna(0)

    # Replace inf values
    features_df = features_df.replace([np.inf, -np.inf], 0)

    print(f"\nFeatures engineered: {features_df.shape[1] - 2} features per session")
    print(f"Total sessions: {len(features_df)}")

    # Store feature names (excluding session_id and user_id)
    feature_names = [col for col in features_df.columns
                     if col not in ['session_id', 'user_id']]

    return features_df, feature_names


def test_multicollinearity(features_df : pd.DataFrame, feature_names : list) -> pd.DataFrame:
    """
    Test for multicollinearity using VIF
    """

    X = features_df[feature_names].fillna(0)

    # VIF calculation
    key_features = [col for col in feature_names if any(
        keyword in col for keyword in ['mean', 'std', 'magnitude', 'count', 'duration']
    )][:50]  # Limit to 50 features

    X_subset = X[key_features]

    # Standardize
    X_scaled = StandardScaler().fit_transform(X_subset)
    X_scaled_df = pd.DataFrame(X_scaled, columns=key_features)

    vif_data = pd.DataFrame()
    vif_data["Feature"] = key_features
    vif_data["VIF"] = [variance_inflation_factor(X_scaled_df.values, i)
                      for i in range(len(key_features))]

    vif_data = vif_data.sort_values('VIF', ascending=False)

    print("\nTop 20 Features by VIF:")
    print(vif_data.head(20))

    print(f"\nFeatures with VIF > 10 (high multicollinearity): {len(vif_data[vif_data['VIF'] > 10])}")
    print(f"Features with VIF > 5: {len(vif_data[vif_data['VIF'] > 5])}")

    # Visualize VIF
    fig = px.bar(vif_data.head(30), x='Feature', y='VIF',
                 title='Variance Inflation Factor (VIF) - Top 30 Features',
                 labels={'VIF': 'VIF Score'},
                 color='VIF',
                 color_continuous_scale='reds')
    fig.add_hline(y=10, line_dash="dash", line_color="red",
                 annotation_text="VIF = 10 (High Multicollinearity)")
    fig.add_hline(y=5, line_dash="dash", line_color="orange",
                 annotation_text="VIF = 5")
    fig.update_xaxes(tickangle=45)
    fig.show()

    return vif_data


def analyze_feature_importance(features_df, feature_names):

    X = features_df[feature_names]
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(features_df['user_id'])

    # 1. Mutual Information
    mi_scores = mutual_info_classif(X, y, random_state=42)
    mi_df = pd.DataFrame({
        'Feature': feature_names,
        'MI_Score': mi_scores
    }).sort_values('MI_Score', ascending=False)

    print("\nTop 20 Features by Mutual Information:")
    print(mi_df.head(20))

    # ANOVA
    f_scores, p_values = f_classif(X, y)
    f_df = pd.DataFrame({
        'Feature': feature_names,
        'F_Score': f_scores,
        'P_Value': p_values
    }).sort_values('F_Score', ascending=False)

    print("\nTop 20 Features by F-score:")
    print(f_df.head(20))

    # Visualization
    fig = make_subplots(rows=1, cols=2,
                       subplot_titles=('Top 20 by Mutual Information',
                                     'Top 20 by F-statistic'))

    top_mi = mi_df.head(20).sort_values('MI_Score')
    fig.add_trace(
        go.Bar(y=top_mi['Feature'], x=top_mi['MI_Score'],
               orientation='h', name='MI Score',
               marker_color='lightblue'),
        row=1, col=1
    )

    top_f = f_df.head(20).sort_values('F_Score')
    fig.add_trace(
        go.Bar(y=top_f['Feature'], x=top_f['F_Score'],
               orientation='h', name='F Score',
               marker_color='lightcoral'),
        row=1, col=2
    )

    fig.update_layout(height=600, showlegend=False,
                     title_text="Feature Importance Analysis")
    fig.show()

    return mi_df, f_df


In [262]:


def select_top_features(mi_df=None, f_df=None, feature_names=None, method= 'mutual_info', n_features=100, verbose=True):
    """
    Select top N features
    """

    if method == 'mutual_info':
        top_features = mi_df.head(n_features)['Feature'].tolist()
        if verbose:
            print(f"\nSelected top {n_features} features by Mutual Information")
            print(f"MI Score range: {mi_df.iloc[0]['MI_Score']:.4f} to {mi_df.iloc[n_features-1]['MI_Score']:.4f}")

    elif method == 'f_statistic':
        top_features = f_df.head(n_features)['Feature'].tolist()
        if verbose:
            print(f"\nSelected top {n_features} features by F-statistic")
            print(f"F-Score range: {f_df.iloc[0]['F_Score']:.4e} to {f_df.iloc[n_features-1]['F_Score']:.4e}")

    elif method == 'combined':
        mi_top = set(mi_df.head(max(1, n_features//2))['Feature'].tolist())
        f_top = set(f_df.head(max(1, n_features//2))['Feature'].tolist())
        top_features = list(mi_top.union(f_top))[:n_features]
        if verbose:
            print(f"\nSelected {len(top_features)} features by combined method")
            print(f"  From MI: {len(mi_top)}, From F-stat: {len(f_top)}")

    else:
        raise ValueError("Unknown method. Use 'mutual_info', 'f_statistic', or 'combined'")

    # Reduction stats
    original_count = len(feature_names)
    reduction_pct = (1 - len(top_features) / original_count) * 100
    print(f"\nFeature reduction: {original_count} → {len(top_features)} ({reduction_pct:.1f}% reduction)")

    # Categories stats
    feature_categories = defaultdict(int)
    for feat in top_features:
        if 'magnitude' in feat:
            feature_categories['Magnitude'] += 1
        elif 'energy' in feat:
            feature_categories['Energy'] += 1
        elif 'ratio' in feat:
            feature_categories['Ratio'] += 1
        elif 'interval' in feat:
            feature_categories['Temporal'] += 1
        elif 'count' in feat:
            feature_categories['Count'] += 1
        elif 'duration' in feat:
            feature_categories['Duration'] += 1
        else:
            feature_categories['Statistical'] += 1

    print("\nSelected features by category:")
    for category, count in sorted(feature_categories.items(), key=lambda x: x[1], reverse=True):
        print(f"  {category}: {count}")

    return top_features


def train_models_with_feature_selection(train_data, feature_names, mi_df, n_features_list=(50, 100, 200), rf_params=None, random_state=42):
    """
    Train LogisticRegression and RandomForest on reduced feature sets
    """

    X_train = train_data["X_train_scaled"]
    X_test = train_data["X_test_scaled"]
    y_train = train_data["y_train"]
    y_test = train_data["y_test"]

    if rf_params is None:
        rf_params = dict(
            n_estimators=100,
            max_depth=20,
            min_samples_split=5,
            min_samples_leaf=2,
            max_features=None,
            class_weight=None,
            bootstrap=True,
            n_jobs=-1,
            random_state=random_state
        )

    feature_selection_results = {}

    for n_features in n_features_list:
        print(f"\nTRAINING WITH TOP {n_features} FEATURES ")

        top_features = mi_df.head(n_features)["Feature"].tolist()
        feature_indices = [feature_names.index(f) for f in top_features]

        X_train_reduced = X_train[:, feature_indices]
        X_test_reduced = X_test[:, feature_indices]

        results_key = f"top_{n_features}"
        feature_selection_results[results_key] = {}

        # Logistic Regression
        start = time.time()

        lr = LogisticRegression(max_iter=2000, random_state=random_state, n_jobs=-1)
        lr.fit(X_train_reduced, y_train)

        train_time = time.time() - start
        y_pred_train = lr.predict(X_train_reduced)
        y_pred_test = lr.predict(X_test_reduced)
        y_pred_proba = lr.predict_proba(X_test_reduced)

        lr_res = evaluate_model(
            lr, f"LogReg_Top{n_features}",
            {"y_train": y_train, "y_test": y_test},
            y_pred_train, y_pred_test, y_pred_proba, train_time
        )
        lr_res["n_features"] = int(n_features)
        feature_selection_results[results_key]["logreg"] = lr_res

        # Random Forest
        start = time.time()

        rf = RandomForestClassifier(**rf_params)
        rf.fit(X_train_reduced, y_train)

        train_time = time.time() - start
        y_pred_train = rf.predict(X_train_reduced)
        y_pred_test = rf.predict(X_test_reduced)
        y_pred_proba = rf.predict_proba(X_test_reduced)

        rf_res = evaluate_model(
            rf, f"RandomForest_Top{n_features}",
            {"y_train": y_train, "y_test": y_test},
            y_pred_train, y_pred_test, y_pred_proba, train_time
        )
        rf_res["n_features"] = int(n_features)
        feature_selection_results[results_key]["random_forest"] = rf_res

    return feature_selection_results


def compare_feature_selection_results(original_results, feature_selection_results, n_original_features):
    '''Feature selectoion comparison'''

    comparison_data = []

    #Original_Results
    for k, res in original_results.items():
        name = res.get("model_name", str(k))
        if ("LogReg" in name) or ("RandomForest" in name) or (k in ("logreg", "random_forest", "rf")):
            model_label = "LogReg" if "Log" in name or k == "logreg" else "RandomForest"
            comparison_data.append({
                "Model": model_label,
                "N_Features": int(n_original_features),
                "Feature_Set": "All Features",
                "Test_Accuracy": res.get("test_accuracy"),
                "Test_F1": res.get("test_f1"),
                "Test_MCC": res.get("test_mcc"),
                "Train_Time": res.get("train_time")
            })

    # Feature selection results
    for key, models_dict in feature_selection_results.items():
        for model_key, res in models_dict.items():
            comparison_data.append({
                "Model": "LogReg" if model_key == "logreg" else "RandomForest",
                "N_Features": int(res["n_features"]),
                "Feature_Set": f"Top {int(res['n_features'])}",
                "Test_Accuracy": res.get("test_accuracy"),
                "Test_F1": res.get("test_f1"),
                "Test_MCC": res.get("test_mcc"),
                "Train_Time": res.get("train_time")
            })

    comparison_df = pd.DataFrame(comparison_data)
    print("\nComparison Table:")
    print(comparison_df.to_string(index=False))

    print("\nFEATURE SELECTION IMPACT ANALYSIS")

    for model_name in comparison_df["Model"].unique():
        model_data = comparison_df[comparison_df["Model"] == model_name].sort_values("N_Features")

        if len(model_data) <= 1:
            continue

        baseline = model_data[model_data["N_Features"] == model_data["N_Features"].max()].iloc[0]

        print(f"\n{model_name}:")
        print(f"  Baseline (all features): Acc={baseline['Test_Accuracy']:.4f}, Time={baseline['Train_Time']:.2f}s")

        for _, row in model_data.iterrows():
            if row["N_Features"] < baseline["N_Features"]:
                acc_change = (row["Test_Accuracy"] - baseline["Test_Accuracy"]) * 100
                time_speedup = (baseline["Train_Time"] / row["Train_Time"] - 1) * 100
                print(f"  Top {row['N_Features']}: Acc={row['Test_Accuracy']:.4f} ({acc_change:+.2f}%), "
                      f"Time={row['Train_Time']:.2f}s ({time_speedup:+.1f}% faster)")

    return comparison_df


def visualize_feature_selection_comparison(comparison_df):
    """
    FS visualisation
    """
    comparison_df = comparison_df.sort_values(by="N_Features")
    fig1 = px.line(comparison_df, x="N_Features", y="Test_Accuracy", color="Model", markers=True,
                   title="Test Accuracy vs Number of Features",
                   labels={"N_Features": "Number of Features", "Test_Accuracy": "Test Accuracy"})
    fig1.update_layout(height=500)
    fig1.show()

    fig2 = px.line(comparison_df, x="N_Features", y="Train_Time", color="Model", markers=True,
                   title="Training Time vs Number of Features",
                   labels={"N_Features": "Number of Features", "Train_Time": "Training Time (s)"})
    fig2.update_layout(height=500)
    fig2.show()

    fig3 = px.scatter(comparison_df, x="Train_Time", y="Test_Accuracy", color="Model", size="N_Features",
                      hover_data=["Feature_Set", "N_Features"],
                      title="Accuracy vs Training Time (size = number of features)",
                      labels={"Train_Time": "Training Time (seconds)", "Test_Accuracy": "Test Accuracy"})
    fig3.update_layout(height=600)
    fig3.show()

    fig4 = go.Figure()
    for model in comparison_df["Model"].unique():
        md = comparison_df[comparison_df["Model"] == model].sort_values("N_Features")
        fig4.add_trace(go.Bar(
            name=model,
            x=[("Top " + str(n)) if fs != "All Features" else "All"
               for n, fs in zip(md["N_Features"], md["Feature_Set"])],
            y=md["Test_F1"],
            text=md["Test_F1"].round(3),
            textposition="auto"
        ))
    fig4.update_layout(title="F1-Score Comparison: Feature Selection Impact",
                       xaxis_title="Feature Set", yaxis_title="Test F1-Score",
                       barmode="group", height=500)
    fig4.show()




## Modeling <a name="Modeling-Functions"></a>

In [239]:
def preprocessing(features_df: pd.DataFrame, feature_names : list, test_size=0.25, stratify=True) -> dict:

  X = features_df[feature_names].values
  #encode users_id
  label_encoder = LabelEncoder()
  y = label_encoder.fit_transform(features_df['user_id'])

  stratify_param = y if stratify else None


  #train test split
  X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=stratify_param
    )

    # Standardize features
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)

  print(f"\nTrain set: {X_train.shape[0]} samples")
  print(f"Test set: {X_test.shape[0]} samples")
  print(f"Number of features: {X_train.shape[1]}")
  print(f"Number of classes (users): {len(np.unique(y))}")

    # Class distribution
  train_dist = pd.Series(y_train).value_counts().sort_index()
  test_dist = pd.Series(y_test).value_counts().sort_index()

  print("\nClass distribution in train set:")
  print(train_dist.describe())
  print("\nClass distribution in test set:")
  print(test_dist.describe())

  return {
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'X_train_scaled': X_train_scaled,
        'X_test_scaled': X_test_scaled,
        'scaler': scaler,
        'label_encoder': label_encoder
    }


def evaluate_model(model, model_name, train_data, y_pred_train, y_pred_test, y_pred_proba, train_time):
    """
    evaluation metrics
    """

    y_train = train_data['y_train']
    y_test = train_data['y_test']

    results = {
        'model_name': model_name,
        'train_time': train_time
    }

    # Accuracy
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    results['train_accuracy'] = train_acc
    results['test_accuracy'] = test_acc

    # Precision, Recall, F1
    train_precision = precision_score(y_train, y_pred_train, average='weighted', zero_division=0)
    test_precision = precision_score(y_test, y_pred_test, average='weighted', zero_division=0)

    train_recall = recall_score(y_train, y_pred_train, average='weighted', zero_division=0)
    test_recall = recall_score(y_test, y_pred_test, average='weighted', zero_division=0)

    train_f1 = f1_score(y_train, y_pred_train, average='weighted', zero_division=0)
    test_f1 = f1_score(y_test, y_pred_test, average='weighted', zero_division=0)

    results['train_precision'] = train_precision
    results['test_precision'] = test_precision
    results['train_recall'] = train_recall
    results['test_recall'] = test_recall
    results['train_f1'] = train_f1
    results['test_f1'] = test_f1

    # Matthews Correlation Coefficient
    train_mcc = matthews_corrcoef(y_train, y_pred_train)
    test_mcc = matthews_corrcoef(y_test, y_pred_test)
    results['train_mcc'] = train_mcc
    results['test_mcc'] = test_mcc

    # Cohen's Kappa
    train_kappa = cohen_kappa_score(y_train, y_pred_train)
    test_kappa = cohen_kappa_score(y_test, y_pred_test)
    results['train_kappa'] = train_kappa
    results['test_kappa'] = test_kappa

    # ROC AUC
    try:
        test_auc = roc_auc_score(y_test, y_pred_proba,
                                multi_class='ovr', average='weighted')
        results['test_auc'] = test_auc
    except:
        results['test_auc'] = None

    # results
    print(f"\nTraining Time: {train_time:.2f} seconds")
    print(f"\nAccuracy:")
    print(f"  Train: {train_acc:.4f}")
    print(f"  Test:  {test_acc:.4f}")

    print(f"\nPrecision (weighted):")
    print(f"  Train: {train_precision:.4f}")
    print(f"  Test:  {test_precision:.4f}")

    print(f"\nRecall (weighted):")
    print(f"  Train: {train_recall:.4f}")
    print(f"  Test:  {test_recall:.4f}")

    print(f"\nF1-Score (weighted):")
    print(f"  Train: {train_f1:.4f}")
    print(f"  Test:  {test_f1:.4f}")

    print(f"\nMatthews Correlation Coefficient:")
    print(f"  Train: {train_mcc:.4f}")
    print(f"  Test:  {test_mcc:.4f}")

    print(f"\nCohen's Kappa:")
    print(f"  Train: {train_kappa:.4f}")
    print(f"  Test:  {test_kappa:.4f}")

    if results['test_auc'] is not None:
        print(f"\nROC AUC:")
        print(f"  Test:  {test_auc:.4f}")

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred_test)
    results['confusion_matrix'] = cm

    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred_test, zero_division=0))

    return results


def train_baseline_model(train_data):
    """
    Logistic Regression
    """

    start_time = time.time()

    # Logistic Regression
    baseline = LogisticRegression(max_iter=100, random_state=4, n_jobs=-1)
    baseline.fit(train_data['X_train_scaled'], train_data['y_train'])

    #Computational time
    train_time = time.time() - start_time

    # Predictions
    y_pred_train = baseline.predict(train_data['X_train_scaled'])
    y_pred_test = baseline.predict(train_data['X_test_scaled'])
    y_pred_proba = baseline.predict_proba(train_data['X_test_scaled'])

    # Evaluate
    results = evaluate_model(baseline, 'Baseline_LogisticRegression',
                            train_data, y_pred_train, y_pred_test, y_pred_proba,
                            train_time)

    return baseline, results

def train_forest_model(train_data):
    """
    Train Random Forest model
    """

    start_time = time.time()

    # Random Forest with hyperparameter tuning

    rf_params = {
        'n_estimators': [100, 200, 300, 500],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2', None],
        'bootstrap': [True, False],
        'class_weight': ['balanced', None]
    }

    rf = RandomForestClassifier(random_state=42, n_jobs=-1)

    rf_grid = RandomizedSearchCV(
        rf,
        rf_params,
        n_iter=20,
        cv=5,
        random_state=42,
        n_jobs=-1,
        verbose=2,
        scoring='accuracy',
        return_train_score=True
    )
    rf_grid.fit(train_data['X_train_scaled'], train_data['y_train'])

    best_rf = rf_grid.best_estimator_

    print(f"\nBest Random Forest parameters:")
    for param, value in rf_grid.best_params_.items():
        print(f"  {param}: {value}")

    print(f"\nBest cross-validation score: {rf_grid.best_score_:.4f}")

    train_time = time.time() - start_time

    # Predictions
    y_pred_train = best_rf.predict(train_data['X_train_scaled'])
    y_pred_test = best_rf.predict(train_data['X_test_scaled'])
    y_pred_proba = best_rf.predict_proba(train_data['X_test_scaled'])

    # Evaluate
    results = evaluate_model(best_rf, 'RandomForest',
                            train_data, y_pred_train, y_pred_test, y_pred_proba,
                            train_time)

    # Add grid search results
    results['best_params'] = rf_grid.best_params_
    results['cv_score'] = rf_grid.best_score_
    results['cv_results'] = rf_grid.cv_results_

    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': range(len(best_rf.feature_importances_)),
        'importance': best_rf.feature_importances_
    }).sort_values('importance', ascending=False)

    print(f"\nTop 10 Most Important Features:")
    print(feature_importance.head(10))

    return best_rf, results


def train_ensemble_model(train_data):
    """
    Train ensemble model here is Random Forest + Gradient Boosting
    """

    start_time = time.time()

    # Random Forest with hyperparameter tuning
    rf_params = {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }

    rf = RandomForestClassifier(random_state=42, n_jobs=-1)
    rf_grid = RandomizedSearchCV(rf, rf_params, n_iter=10, cv=3,
                                 random_state=42, n_jobs=-1, verbose=1)
    rf_grid.fit(train_data['X_train_scaled'], train_data['y_train'])

    best_rf = rf_grid.best_estimator_

    # Gradient Boosting
    gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
    gb.fit(train_data['X_train_scaled'], train_data['y_train'])

    # Voting Classifier
    voting_clf = VotingClassifier(
        estimators=[('rf', best_rf), ('gb', gb)],
        voting='soft',
        n_jobs=-1
    )
    voting_clf.fit(train_data['X_train_scaled'], train_data['y_train'])

    train_time = time.time() - start_time

    # Predictions
    y_pred_train = voting_clf.predict(train_data['X_train_scaled'])
    y_pred_test = voting_clf.predict(train_data['X_test_scaled'])
    y_pred_proba = voting_clf.predict_proba(train_data['X_test_scaled'])

    # Evaluate
    results = evaluate_model(voting_clf, 'Ensemble_RF+GB',
                            train_data, y_pred_train, y_pred_test, y_pred_proba,
                            train_time)

    return voting_clf, results


def train_boosting_model(train_data):
    """
    Train XGBoost model
    Best params by search
    """
    start_time = time.time()

    xgb_params = {
            'n_estimators': [100, 200],
            'max_depth': [5, 10],
            'learning_rate': [0.01, 0.1],
            'subsample': [0.8, 1.0]
        }

    xgb = XGBClassifier(random_state=42, n_jobs=-1, eval_metric='mlogloss')
    xgb_grid = RandomizedSearchCV(xgb, xgb_params, n_iter=8, cv=3,
                                     random_state=42, n_jobs=-1, verbose=1)
    xgb_grid.fit(train_data['X_train_scaled'], train_data['y_train'])

    best_xgb = xgb_grid.best_estimator_
    print(f"Best XGBoost parameters: {xgb_grid.best_params_}")

    train_time = time.time() - start_time

    # Predictions
    y_pred_train = best_xgb.predict(train_data['X_train_scaled'])
    y_pred_test = best_xgb.predict(train_data['X_test_scaled'])
    y_pred_proba = best_xgb.predict_proba(train_data['X_test_scaled'])

    # Evaluate
    results = evaluate_model(best_xgb, 'Boosting_XGBoost',
                            train_data, y_pred_train, y_pred_test, y_pred_proba,
                            train_time)

    return best_xgb, results


def train_stacking_model(train_data):
    """
    Train stacking model here is Random Forest and Gradient Boosting + XGBoosting as baselearners
    and meta learner is baseline
    """

    start_time = time.time()

    # Base learners
    base_learners = [
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)),
        ('gb', GradientBoostingClassifier(n_estimators=50, random_state=42)),
        ('xgb', XGBClassifier(n_estimators=100, random_state=42, eval_metric='mlogloss'))
    ]

    # Meta learner
    meta_learner = LogisticRegression(max_iter=1000, random_state=42)

    # Stacking classifier
    stacking_clf = StackingClassifier(
        estimators=base_learners,
        final_estimator=meta_learner,
        cv = 3,
        n_jobs=-1
    )
    #Train
    stacking_clf.fit(train_data['X_train_scaled'], train_data['y_train'])

    train_time = time.time() - start_time

    # Predictions
    y_pred_train = stacking_clf.predict(train_data['X_train_scaled'])
    y_pred_test = stacking_clf.predict(train_data['X_test_scaled'])
    y_pred_proba = stacking_clf.predict_proba(train_data['X_test_scaled'])

    # Evaluate
    results = evaluate_model(stacking_clf, 'Stacking_Ensemble',
                            train_data, y_pred_train, y_pred_test, y_pred_proba,
                            train_time)

    return stacking_clf, results


def train_neural_network(train_data):
    """
    Train NN model
    """

    start_time = time.time()

    # Build model
    n_classes = len(np.unique(train_data['y_train']))
    n_features = train_data['X_train_scaled'].shape[1]

    model = keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(n_features,)),
        layers.Dropout(0.3),
        layers.BatchNormalization(),

        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.BatchNormalization(),

        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.BatchNormalization(),

        layers.Dense(32, activation='relu'),
        layers.Dropout(0.2),

        layers.Dense(n_classes, activation='softmax')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    print("\nModel Summary:")
    model.summary()

    # earlystopping
    early_stop = callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    reduce_lr = callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6
    )

    # Train
    history = model.fit(
        train_data['X_train_scaled'], train_data['y_train'],
        validation_split=0.2,
        epochs=100,
        batch_size=32,
        callbacks=[early_stop, reduce_lr],
        verbose=1
    )

    train_time = time.time() - start_time

    # Predictions
    y_pred_train = np.argmax(model.predict(train_data['X_train_scaled']), axis=1)
    y_pred_test = np.argmax(model.predict(train_data['X_test_scaled']), axis=1)
    y_pred_proba = model.predict(train_data['X_test_scaled'])

    # Evaluate
    results = evaluate_model(model, 'NeuralNetwork',
                            train_data, y_pred_train, y_pred_test, y_pred_proba,
                            train_time)

    # Add training history
    results['history'] = history.history

    # Visualize training
    _visualize_nn_training(history)

    return model, results


def _visualize_nn_training(history):
    """
    Visualizations for Neural Networks
    """
    fig = make_subplots(rows=1, cols=2,
                       subplot_titles=('Model Loss', 'Model Accuracy'))

    # Loss
    fig.add_trace(
        go.Scatter(y=history.history['loss'], mode='lines', name='Train Loss'),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(y=history.history['val_loss'], mode='lines', name='Val Loss'),
        row=1, col=1
    )

    # Accuracy
    fig.add_trace(
        go.Scatter(y=history.history['accuracy'], mode='lines', name='Train Acc'),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(y=history.history['val_accuracy'], mode='lines', name='Val Acc'),
        row=1, col=2
    )

    fig.update_xaxes(title_text="Epoch", row=1, col=1)
    fig.update_xaxes(title_text="Epoch", row=1, col=2)
    fig.update_yaxes(title_text="Loss", row=1, col=1)
    fig.update_yaxes(title_text="Accuracy", row=1, col=2)

    fig.update_layout(height=400, title_text="Neural Network Training History")
    fig.show()


def compare_all_models(all_results : dict)-> pd.DataFrame:
    """
    Compare all models results
    """

    comparison_df = pd.DataFrame([
        {
            'Model': results['model_name'],
            'Test Accuracy': results['test_accuracy'],
            'Test F1': results['test_f1'],
            'Test Precision': results['test_precision'],
            'Test Recall': results['test_recall'],
            'Test MCC': results['test_mcc'],
            'Test Kappa': results['test_kappa'],
            'Test AUC': results.get('test_auc', np.nan),
            'Train Time (s)': results['train_time']
        }
        for results in all_results.values()
    ]).sort_values('Test Accuracy', ascending=False)

    print("\nModel Performance Summary:")
    print(comparison_df.to_string(index=False))

    # Visualization
    metrics = ['Test Accuracy', 'Test F1', 'Test Precision', 'Test Recall', 'Test MCC']

    fig = go.Figure()

    for metric in metrics:
        fig.add_trace(go.Bar(
            name=metric,
            x=comparison_df['Model'],
            y=comparison_df[metric],
            text=comparison_df[metric].round(4),
            textposition='auto'
        ))

    fig.update_layout(
        title='Model Performance Comparison',
        xaxis_title='Model',
        yaxis_title='Score',
        barmode='group',
        height=500,
        showlegend=True
    )
    fig.show()

    # Training time
    fig2 = px.bar(comparison_df, x='Model', y='Train Time (s)',
                 title='Training Time Comparison',
                 color='Train Time (s)',
                 color_continuous_scale='viridis',
                 text='Train Time (s)')
    fig2.update_traces(texttemplate='%{text:.2f}s', textposition='outside')
    fig2.show()

  # Confusion matrix
    n_models = min(len(all_results), 4)
    fig3 = make_subplots(
        rows=2, cols=2,
        subplot_titles=[f"{res['model_name']}" for res in list(all_results.values())[:4]],
        specs=[[{'type': 'heatmap'}, {'type': 'heatmap'}],
               [{'type': 'heatmap'}, {'type': 'heatmap'}]]
    )

    positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

    for idx, (model_key, results) in enumerate(list(all_results.items())[:4]):
        cm = results['confusion_matrix']
        row, col = positions[idx]

        fig3.add_trace(
            go.Heatmap(
                z=cm,
                colorscale='Blues',
                showscale=(idx == 0),
                text=cm,
                texttemplate='%{text}',
                textfont={"size": 8}
            ),
            row=row, col=col
        )

    fig3.update_layout(height=800, title_text="Confusion Matrices Comparison")
    fig3.show()


    return comparison_df


In [226]:
def train_ResMLP(train_data, use_class_weights=True, seed=42):
    """
    NN + Residual MLP blocks
    """

    start_time = time.time()

    # Reproducibility
    tf.random.set_seed(seed)
    np.random.seed(seed)

    X_train = train_data['X_train_scaled']
    y_train = train_data['y_train']
    X_test = train_data['X_test_scaled']

    n_classes = len(np.unique(y_train))
    n_features = X_train.shape[1]

    #weights
    class_weight = None
    if use_class_weights:
        counts = np.bincount(y_train, minlength=n_classes)
        # weights ~ inverse frequency (мягко)
        weights = counts.sum() / (n_classes * np.maximum(counts, 1))
        class_weight = {i: float(w) for i, w in enumerate(weights)}
        print("[NN] Using class_weight:", class_weight)

    #residual MLP
    wd = 1e-4  # weight decay / L2-ish
    dropout = 0.25

    inputs = keras.Input(shape=(n_features,))
    x = layers.Dense(256, activation=None, kernel_regularizer=regularizers.l2(wd))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(dropout)(x)

    def res_block(x, units, dropout_rate=0.2):
        shortcut = x
        # Project shortcut if needed
        if x.shape[-1] != units:
            shortcut = layers.Dense(units, activation=None, kernel_regularizer=regularizers.l2(wd))(shortcut)

        h = layers.Dense(units, activation=None, kernel_regularizer=regularizers.l2(wd))(x)
        h = layers.BatchNormalization()(h)
        h = layers.Activation("relu")(h)
        h = layers.Dropout(dropout_rate)(h)

        h = layers.Dense(units, activation=None, kernel_regularizer=regularizers.l2(wd))(h)
        h = layers.BatchNormalization()(h)

        out = layers.Add()([shortcut, h])
        out = layers.Activation("relu")(out)
        return out

    x = res_block(x, 256, 0.25)
    x = res_block(x, 128, 0.20)
    x = res_block(x, 128, 0.20)
    x = res_block(x, 64,  0.15)

    x = layers.Dense(64, activation="relu", kernel_regularizer=regularizers.l2(wd))(x)
    x = layers.Dropout(0.15)(x)

    outputs = layers.Dense(n_classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    optimizer = keras.optimizers.AdamW(learning_rate=3e-4, weight_decay=1e-4)

    loss_fn = keras.losses.SparseCategoricalCrossentropy()

    model.compile(
        optimizer=optimizer,
        loss=loss_fn,
        metrics=["accuracy"]
    )

    print("\nModel Summary:")
    model.summary()

    # earlystop
    early_stop = callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=15,
        mode="max",
        restore_best_weights=True
    )

    reduce_lr = callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=6,
        min_lr=1e-6,
        verbose=1
    )

    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=200,
        batch_size=64,              # часто лучше для BN
        callbacks=[early_stop, reduce_lr],
        class_weight=class_weight,
        verbose=1
    )

    train_time = time.time() - start_time

    y_pred_train = np.argmax(model.predict(X_train, verbose=0), axis=1)
    y_pred_test  = np.argmax(model.predict(X_test,  verbose=0), axis=1)
    y_pred_proba = model.predict(X_test, verbose=0)

    results = evaluate_model(
        model, "NeuralNetwork+ResMLP",
        train_data, y_pred_train, y_pred_test, y_pred_proba,
        train_time
    )

    results["history"] = history.history
    _visualize_nn_training(history)

    return model, results


## PCA

In [229]:

def analyze_pca(train_data, feature_names=None, n_components_range=(0.80, 0.90, 0.95, 0.99), random_state=42):
    """
    PCA analysis
    """
    X_train = train_data["X_train_scaled"]

    pca_full = PCA(random_state=random_state)
    pca_full.fit(X_train)

    cumsum_variance = np.cumsum(pca_full.explained_variance_ratio_)

    thresholds_info = []
    n_total = X_train.shape[1]
    for thr in n_components_range:
        n_comp = int(np.argmax(cumsum_variance >= thr) + 1)
        thresholds_info.append({
            "variance_threshold": float(thr),
            "n_components": n_comp,
            "pct_of_original_features": float((n_comp / n_total) * 100.0),
        })

    print("\nVariance Explained Analysis:")
    for info in thresholds_info:
        print(f"  {info['variance_threshold']*100:.0f}% variance: {info['n_components']} components "
              f"({info['pct_of_original_features']:.1f}% of original features)")

    return {
        "pca_full": pca_full,
        "cumsum_variance": cumsum_variance,
        "thresholds_info": thresholds_info,
        "feature_names": feature_names
    }


def build_pca_configs(train_data, n_components_range=(0.80, 0.90, 0.95, 0.99), random_state=42):
    """
    Fit PCA for train and test
    """
    X_train = train_data["X_train_scaled"]
    X_test = train_data["X_test_scaled"]

    pca_full = PCA(random_state=random_state)
    pca_full.fit(X_train)
    cumsum_variance = np.cumsum(pca_full.explained_variance_ratio_)

    pca_configs = {}
    for thr in n_components_range:
        n_comp = int(np.argmax(cumsum_variance >= thr) + 1)

        pca = PCA(n_components=n_comp, random_state=random_state)
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)

        pca_configs[thr] = {
            "pca": pca,
            "X_train": X_train_pca,
            "X_test": X_test_pca,
            "n_components": n_comp,
            "variance_explained": float(pca.explained_variance_ratio_.sum())
        }

    return pca_configs


def train_models_with_pca(train_data, pca_configs, variance_thresholds=(0.90, 0.95), random_state=42):
    """
    Train model with PCA
    """
    y_train = train_data["y_train"]
    y_test = train_data["y_test"]

    pca_results = {}

    for thr in variance_thresholds:
        if thr not in pca_configs:
            print(f"[PCA] config for {thr} not found, skipping.")
            continue

        cfg = pca_configs[thr]
        X_train_pca = cfg["X_train"]
        X_test_pca = cfg["X_test"]
        n_components = cfg["n_components"]

        key = f"pca_{int(thr * 100)}"
        pca_results[key] = {}

        print("\n" + "=" * 60)
        print(f"MODELS WITH PCA ({thr*100:.0f}% variance) | components={n_components} "
              f"| explained={cfg['variance_explained']:.4f}")
        print("=" * 60)

        # Logistic Regression
        start = time.time()
        lr = LogisticRegression(max_iter=2000, random_state=random_state, n_jobs=-1)
        lr.fit(X_train_pca, y_train)
        train_time = time.time() - start

        y_pred_train = lr.predict(X_train_pca)
        y_pred_test = lr.predict(X_test_pca)
        y_pred_proba = lr.predict_proba(X_test_pca)

        res = evaluate_model(
            lr, f"Baseline_LR_PCA{int(thr*100)}",
            {"y_train": y_train, "y_test": y_test},
            y_pred_train, y_pred_test, y_pred_proba,
            train_time
        )
        res.update({"n_components": n_components, "variance_threshold": thr})
        pca_results[key]["baseline_lr"] = res

        # Random Forest
        start = time.time()
        rf = RandomForestClassifier(
            n_estimators=300, max_depth=20,
            random_state=random_state, n_jobs=-1
        )
        rf.fit(X_train_pca, y_train)
        train_time = time.time() - start

        y_pred_train = rf.predict(X_train_pca)
        y_pred_test = rf.predict(X_test_pca)
        y_pred_proba = rf.predict_proba(X_test_pca)

        res = evaluate_model(
            rf, f"RandomForest_PCA{int(thr*100)}",
            {"y_train": y_train, "y_test": y_test},
            y_pred_train, y_pred_test, y_pred_proba,
            train_time
        )
        res.update({"n_components": n_components, "variance_threshold": thr})
        pca_results[key]["random_forest"] = res

        # XGBoost
        try:
            start = time.time()
            xgb = XGBClassifier(
                n_estimators=600, max_depth=8, learning_rate=0.05,
                subsample=0.9, colsample_bytree=0.9,
                random_state=random_state, n_jobs=-1,
                eval_metric="mlogloss"
            )
            xgb.fit(X_train_pca, y_train)
            train_time = time.time() - start

            y_pred_train = xgb.predict(X_train_pca)
            y_pred_test = xgb.predict(X_test_pca)
            y_pred_proba = xgb.predict_proba(X_test_pca)

            res = evaluate_model(
                xgb, f"XGBoost_PCA{int(thr*100)}",
                {"y_train": y_train, "y_test": y_test},
                y_pred_train, y_pred_test, y_pred_proba,
                train_time
            )
            res.update({"n_components": n_components, "variance_threshold": thr})
            pca_results[key]["xgboost"] = res
        except Exception as e:
            print(f"[PCA] XGBoost skipped: {e}")

        # Neural Net
        try:
            start = time.time()
            n_classes = int(len(np.unique(y_train)))

            nn = keras.Sequential([
                layers.Dense(128, activation="relu", input_shape=(n_components,)),
                layers.BatchNormalization(),
                layers.Dropout(0.35),

                layers.Dense(64, activation="relu"),
                layers.BatchNormalization(),
                layers.Dropout(0.25),

                layers.Dense(n_classes, activation="softmax")
            ])

            nn.compile(
                optimizer=keras.optimizers.Adam(learning_rate=3e-4),
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"]
            )

            es = callbacks.EarlyStopping(
                monitor="val_accuracy", patience=12,
                mode="max", restore_best_weights=True
            )
            rlrop = callbacks.ReduceLROnPlateau(
                monitor="val_loss", factor=0.5,
                patience=6, min_lr=1e-6, verbose=1
            )

            hist = nn.fit(
                X_train_pca, y_train,
                validation_split=0.2,
                epochs=150,
                batch_size=64,
                callbacks=[es, rlrop],
                verbose=0
            )

            train_time = time.time() - start

            y_pred_train = np.argmax(nn.predict(X_train_pca, verbose=0), axis=1)
            y_pred_test = np.argmax(nn.predict(X_test_pca, verbose=0), axis=1)
            y_pred_proba = nn.predict(X_test_pca, verbose=0)

            res = evaluate_model(
                nn, f"NeuralNet_PCA{int(thr*100)}",
                {"y_train": y_train, "y_test": y_test},
                y_pred_train, y_pred_test, y_pred_proba,
                train_time
            )
            res.update({
                "n_components": n_components,
                "variance_threshold": thr,
                "history": hist.history
            })
            pca_results[key]["neural_net"] = res

        except Exception as e:
            print(f"[PCA] NeuralNet skipped: {e}")

    return pca_results


def compare_pca_vs_original(original_results, pca_results, n_original_features):
    """
    Comparison of results between pca and without
    """
    rows = []

    # Original results
    for k, res in original_results.items():
        rows.append({
            "Model": res.get("model_name", str(k)),
            "Feature_Set": "Original",
            "N_Features": n_original_features,
            "Test_Accuracy": res.get("test_accuracy"),
            "Test_F1": res.get("test_f1"),
            "Test_MCC": res.get("test_mcc"),
            "Train_Time": res.get("train_time")
        })

    # PCA results
    for pca_key, models in pca_results.items():
        for mk, res in models.items():
            rows.append({
                "Model": res.get("model_name", str(mk)),
                "Feature_Set": f"PCA_{int(res['variance_threshold']*100)}%",
                "N_Features": res.get("n_components"),
                "Test_Accuracy": res.get("test_accuracy"),
                "Test_F1": res.get("test_f1"),
                "Test_MCC": res.get("test_mcc"),
                "Train_Time": res.get("train_time")
            })

    comparison_df = pd.DataFrame(rows)

    print(comparison_df.to_string(index=False))

    return comparison_df


In [248]:
def print_pca_summary_statistics(comparison_df, n_original_features):

    summary = comparison_df.groupby('Feature_Set').agg({
        'Test_Accuracy': ['mean', 'std', 'max'],
        'Test_F1': ['mean', 'std', 'max'],
        'Train_Time': ['mean', 'std', 'min']
    }).round(4)

    print("\nPerformance by Feature Set:")
    print(summary)

    print("\nPCA BENEFITS ANALYSIS")

    original_avg_time = comparison_df.loc[comparison_df['Feature_Set'] == 'Original', 'Train_Time'].mean()
    original_avg_acc  = comparison_df.loc[comparison_df['Feature_Set'] == 'Original', 'Test_Accuracy'].mean()

    for feature_set in comparison_df['Feature_Set'].unique():
        if feature_set == 'Original':
            continue

        pca_data = comparison_df[comparison_df['Feature_Set'] == feature_set]
        if pca_data.empty:
            continue

        pca_avg_time = pca_data['Train_Time'].mean()
        pca_avg_acc  = pca_data['Test_Accuracy'].mean()

        pca_n_features = int(pca_data['N_Features'].iloc[0])

        # speedup in %
        speedup = (original_avg_time / pca_avg_time - 1) * 100 if pca_avg_time > 0 else np.inf
        # accuracy change in %
        acc_change = (pca_avg_acc / original_avg_acc - 1) * 100 if original_avg_acc > 0 else np.nan
        # feature reduction in %
        feature_reduction = (1 - (pca_n_features / n_original_features)) * 100 if n_original_features > 0 else np.nan

        print(f"\n{feature_set}:")
        print(f"  Feature reduction: {feature_reduction:.1f}%")
        print(f"  Average speedup: {speedup:+.1f}%")
        print(f"  Average accuracy change: {acc_change:+.2f}%")
        print(f"  Components used: {pca_n_features}")

In [249]:


def visualize_pca_analysis(pca_full, cumsum_variance, thresholds=(0.80, 0.90, 0.95, 0.99), feature_names=None, top_k=10, y_train=None, max_samples=1000, random_state=42):

    np.random.seed(random_state)

    fig = make_subplots(rows=1, cols=2, subplot_titles=("Scree Plot - Explained Variance per Component", "Cumulative Explained Variance"))

    n_components_to_show = min(50, len(pca_full.explained_variance_ratio_))

    fig.add_trace(go.Bar(x=list(range(1, n_components_to_show + 1)), y=pca_full.explained_variance_ratio_[:n_components_to_show], name="Individual Variance"), row=1, col=1)
    fig.add_trace(go.Scatter(x=list(range(1, n_components_to_show + 1)), y=cumsum_variance[:n_components_to_show], mode="lines+markers", name="Cumulative Variance"), row=1, col=2)

    colors = ["red", "orange", "green", "purple", "brown", "gray"]
    for thr, color in zip(thresholds, colors):
        fig.add_hline(y=thr, line_dash="dash", line_color=color, annotation_text=f"{int(thr*100)}%", row=1, col=2)

    fig.update_xaxes(title_text="Principal Component", row=1, col=1)
    fig.update_xaxes(title_text="Number of Components", row=1, col=2)
    fig.update_yaxes(title_text="Explained Variance Ratio", row=1, col=1)
    fig.update_yaxes(title_text="Cumulative Variance", row=1, col=2)
    fig.update_layout(height=500, showlegend=False, title_text="PCA Variance Analysis")
    fig.show()

    #Top feature loadings with dropdown
    if feature_names is not None:
        n_top_components = min(10, len(pca_full.components_))
        fig2 = go.Figure()

        for i in range(n_top_components):
            component = pca_full.components_[i]
            top_idx = np.argsort(np.abs(component))[-top_k:][::-1]
            top_feats = [feature_names[j] for j in top_idx]
            top_vals = component[top_idx]

            fig2.add_trace(go.Bar(name=f"PC{i+1}", x=top_feats, y=top_vals, visible=(i == 0)))

        buttons = []
        for i in range(n_top_components):
            visible = [False] * n_top_components
            visible[i] = True
            buttons.append(dict(label=f"PC{i+1}", method="update", args=[{"visible": visible}, {"title": f"Top {top_k} Feature Loadings for PC{i+1}"}]))

        fig2.update_layout(updatemenus=[dict(active=0, buttons=buttons, direction="down", showactive=True, x=0.17, xanchor="left", y=1.15, yanchor="top")],
                           title=f"Top {top_k} Feature Loadings for PC1",
                           xaxis_title="Feature", yaxis_title="Loading Value", height=600)
        fig2.update_xaxes(tickangle=45)
        fig2.show()
    if y_train is not None:
        pca_3 = PCA(n_components=min(3, pca_full.n_components_), random_state=random_state)
    return


def visualize_pca_projection(X_train_scaled, y_train, random_state=42, max_samples=1000):
    """
   PCA scatter
    """
    import numpy as np
    import plotly.express as px
    from sklearn.decomposition import PCA

    np.random.seed(random_state)

    X = X_train_scaled
    y = y_train

    if len(X) > max_samples:
        idx = np.random.choice(len(X), max_samples, replace=False)
        X = X[idx]
        y = y[idx]

    pca_3 = PCA(n_components=3, random_state=random_state)
    X_pca = pca_3.fit_transform(X)

    # 2D
    fig2d = px.scatter(x=X_pca[:, 0], y=X_pca[:, 1], color=y.astype(str),
                      title="2D PCA Projection (PC1 vs PC2)", labels={"x": "PC1", "y": "PC2", "color": "Class"}, opacity=0.6)
    fig2d.show()

    # 3D
    fig3d = px.scatter_3d(x=X_pca[:, 0], y=X_pca[:, 1], z=X_pca[:, 2], color=y.astype(str),
                          title="3D PCA Projection (PC1, PC2, PC3)", labels={"x": "PC1", "y": "PC2", "z": "PC3", "color": "Class"}, opacity=0.6)
    fig3d.show()

    return pca_3


#Exploratory data analysis <a name="EDA"></a>

In [ ]:
print('Basic data information')
df = load_and_clean_data(df)
df_info(df)

Basic data information
Data loaded: 2318350 rows, 13 columns

DF Information

Dataset Shape: (2318350, 13)

Column Data Types:
uid             object
timestamp        int64
sensor_type      int64
field_0        float64
field_1        float64
field_2        float64
field_3        float64
field_4        float64
field_5        float64
field_6        float64
field_7        float64
session_id      object
user_id         object
dtype: object

Missing Values:
         Missing_Count  Percentage
field_5        2004487   86.461794
field_6        2004487   86.461794
field_7        2004487   86.461794
field_3        2004466   86.460888
field_4        2004466   86.460888

Basic Statistics:
          timestamp   sensor_type       field_0       field_1       field_2  \
count  2.318350e+06  2.318350e+06  2.318350e+06  2.318350e+06  2.318350e+06   
mean   1.752688e+12  5.348974e+00  4.279186e-01 -1.389798e+00 -1.254254e+00   
std    2.950336e+09  5.633553e+00  2.023063e+01  8.308913e+01  3.459845e+01  

Visualisation

In [ ]:
visualize_data(df)

Statistical analysis

In [ ]:
analyze_sensor_signals(df, sample_size=5000)


Gyroscope (Type 1):
  Total events: 333450

  field_0:
    Mean: 0.0005, Std: 0.2037
    Min: -6.9089, Max: 4.8860
    Skewness: -0.5371, Kurtosis: 49.6070
    Normality test (D'Agostino): p-value = 8.7704e-66

  field_1:
    Mean: -0.0002, Std: 0.2391
    Min: -10.4996, Max: 17.2325
    Skewness: 1.1194, Kurtosis: 159.6264
    Normality test (D'Agostino): p-value = 6.1372e-70

  field_2:
    Mean: -0.0005, Std: 0.1525
    Min: -6.4703, Max: 5.5518
    Skewness: -0.4325, Kurtosis: 158.5769
    Normality test (D'Agostino): p-value = 9.3953e-270

Accelerometer (Type 2):
  Total events: 741521

  field_0:
    Mean: 0.1034, Std: 1.4316
    Min: -44.6806, Max: 67.9810
    Skewness: 0.7143, Kurtosis: 17.6598
    Normality test (D'Agostino): p-value = 2.1092e-27

  field_1:
    Mean: 5.8001, Std: 2.1681
    Min: -24.3347, Max: 25.2215
    Skewness: -0.2310, Kurtosis: 0.1999
    Normality test (D'Agostino): p-value = 1.2634e-01

  field_2:
    Mean: 6.8086, Std: 3.0549
    Min: -25.7951, Max:

Autocorrelation analysis

In [ ]:
test_autocorrelation(df, max_lags=50)


Analyzing autocorrelation for session: 6c92cd0b-027c-4a0c-9802-4af636918681

Gyroscope (Type 1):
  Total events: 996

  X-axis (field_0):
    Ljung-Box test:
      Lag 10: p-value = 7.9962e-31
      Lag 20: p-value = 1.0363e-27
    Augmented Dickey-Fuller test:
      Test statistic: -13.7783
      p-value: 9.4012e-26
      Stationary: Yes
    Signal characteristics:
      Mean: -0.0011
      Std: 0.3080
      Autocorr at lag 1: 0.3264
      Autocorr at lag 10: 0.0213

  Y-axis (field_1):
    Ljung-Box test:
      Lag 10: p-value = 1.3032e-40
      Lag 20: p-value = 3.2012e-37
    Augmented Dickey-Fuller test:
      Test statistic: -14.6866
      p-value: 3.0964e-27
      Stationary: Yes
    Signal characteristics:
      Mean: -0.0018
      Std: 0.2195
      Autocorr at lag 1: 0.3411
      Autocorr at lag 10: 0.0037

  Z-axis (field_2):
    Ljung-Box test:
      Lag 10: p-value = 1.7514e-15
      Lag 20: p-value = 1.9201e-13
    Augmented Dickey-Fuller test:
      Test statistic: -15.9

# Feature Enegiring <a name="Feature-Engineering"></a>

In [ ]:
# features creation
features_df, feature_names = engineer_features(df)


Features engineered: 516 features per session
Total sessions: 300


In [ ]:
# Test multicollinearity
vif_data = test_multicollinearity(features_df, feature_names)


Top 20 Features by VIF:
                      Feature         VIF
1                 event_count         inf
3             Gyroscope_count         inf
5               Gravity_count         inf
4         Accelerometer_count         inf
7              Rotation_count         inf
6          Magnetometer_count         inf
8      UserAcceleration_count         inf
47        Gravity_field0_mean  257.218037
28  Accelerometer_field0_mean  255.528325
12    Gyroscope_magnitude_std  213.447465
15       Gyroscope_field1_std  128.331662
11   Gyroscope_magnitude_mean   75.827884
10       Gyroscope_field0_std   65.149367
29   Accelerometer_field0_std   45.923460
48         Gravity_field0_std   37.438303
17       Gyroscope_field2_std   29.333108
13    Gyroscope_magnitude_max    7.832846
0            session_duration    6.642294
35  Accelerometer_field2_mean    6.146869
33  Accelerometer_field1_mean    5.825265

Features with VIF > 10 (high multicollinearity): 16
Features with VIF > 5: 21


In [ ]:
# Analyze feature importance
mi_df, f_df = analyze_feature_importance(features_df, feature_names)


Top 20 Features by Mutual Information:
                          Feature  MI_Score
2              avg_event_interval  2.025587
11             Magnetometer_ratio  1.835202
9                   Gravity_ratio  1.681979
5                 Gyroscope_ratio  1.661759
15         UserAcceleration_ratio  1.638549
110  Accelerometer_magnitude_mean  1.601202
7             Accelerometer_ratio  1.596357
13                 Rotation_ratio  1.587336
194        Gravity_magnitude_mean  1.419141
394           Rotation_field4_min  1.249387
395           Rotation_field4_max  1.248838
392          Rotation_field4_mean  1.239837
396        Rotation_field4_median  1.220700
397           Rotation_field4_q25  1.196316
196         Gravity_magnitude_max  1.148230
398           Rotation_field4_q75  1.139604
3              std_event_interval  0.976934
278   Magnetometer_magnitude_mean  0.971550
195         Gravity_magnitude_std  0.951909
378           Rotation_field2_q75  0.855850

Top 20 Features by F-score:
       

# Cluster Analysis <a name="Cluster-Analysis"></a>


User Clusterisation

In [ ]:
cluster_results = analyze_user_clusters(features_df, feature_names)


Performing t-SNE dimensionality reduction...



Clustering Quality Metrics:
  k=2: Silhouette=0.2696, ARI=0.0049
  k=3: Silhouette=0.2868, ARI=0.0051
  k=4: Silhouette=0.0700, ARI=0.0454
  k=5: Silhouette=0.0604, ARI=0.0401
  k=6: Silhouette=0.0730, ARI=0.0891
  k=7: Silhouette=0.0672, ARI=0.0947
  k=8: Silhouette=0.0802, ARI=0.1371
  k=9: Silhouette=0.0767, ARI=0.1190
  k=10: Silhouette=0.0706, ARI=0.1422


In [ ]:
# Profileusers
extreme_users = profile_extreme_users(features_df, feature_names)


 Most Active User: e2ec300d-bbd6-4d29-a7c8-505970601891
   Average events per session: 20265
   Average session duration: 111.54s

 Most Static User: 1e6e3a87-2cb4-4a76-96c1-d7aebfaa7808
   Average accelerometer energy: 62987.49
   Average magnitude: 9.7444

 Most Dynamic User: 88ebe9b3-59ff-4273-a240-c1274705e361
   Gyroscope std: 0.3613
   Average event interval: 15.63ms


In [ ]:
# Analyze user consistency
user_cv = analyze_user_consistency(features_df, feature_names)


Coefficient of Variation by User:
Lower CV = Higher consistency

Mean CV across features:
user_id
c7825901-6986-4455-8e12-2982e8fb69b2    0.105423
1dec6f98-1792-47c5-8728-37626bc438fb    0.155684
ea6b88b1-d76a-4605-adfe-30b547fc5156    0.174981
88ebe9b3-59ff-4273-a240-c1274705e361    0.181866
513c6adf-7837-4d98-aff4-cfc1de9a1068    0.216306
e98939a8-40b0-495c-8cbe-28be80244a13    0.226797
849add86-83e4-4d63-b81c-81438a31cc34    0.227646
e2ec300d-bbd6-4d29-a7c8-505970601891    0.228583
eef90a36-6643-418b-90a3-9b9d918d9adf    0.230280
c7186a80-9e05-4819-83f4-b19d16643823    0.230797
6ad303e2-e085-4695-8db7-c229e04c3f12    0.232336
301eb109-b1fb-45b2-aae2-929c0886d820    0.246456
6d9e8ecf-66eb-4e02-a287-9fedda296947    0.252851
f930cf60-379d-492a-a524-f9e3b4479c2e    0.284989
1e6e3a87-2cb4-4a76-96c1-d7aebfaa7808    0.331317
98f3a20d-d5fb-4de7-ac64-36befd362cec    0.377230
43b022a4-fc47-42db-9f62-889ddaee7754    0.416701
39390015-0817-419e-a20b-e7326ac1bc1b    0.475489
c05b7e4d-2104-4078-

# Modeling <a name="Modeling"></a>

## Models Training  <a name="Models-Training"></a>

In [241]:
# Prepare train-test split
train_data = preprocessing(features_df, feature_names, test_size=0.25, stratify=False)


Train set: 225 samples
Test set: 75 samples
Number of features: 516
Number of classes (users): 20

Class distribution in train set:
count    20.000000
mean     11.250000
std       1.551739
min       8.000000
25%      10.750000
50%      11.000000
75%      12.250000
max      14.000000
Name: count, dtype: float64

Class distribution in test set:
count    20.000000
mean      3.750000
std       1.551739
min       1.000000
25%       2.750000
50%       4.000000
75%       4.250000
max       7.000000
Name: count, dtype: float64


Logistic Regression

In [242]:
baseline_model, baseline_results = train_baseline_model(train_data)


Training Time: 0.15 seconds

Accuracy:
  Train: 1.0000
  Test:  0.8667

Precision (weighted):
  Train: 1.0000
  Test:  0.8938

Recall (weighted):
  Train: 1.0000
  Test:  0.8667

F1-Score (weighted):
  Train: 1.0000
  Test:  0.8630

Matthews Correlation Coefficient:
  Train: 1.0000
  Test:  0.8601

Cohen's Kappa:
  Train: 1.0000
  Test:  0.8585

ROC AUC:
  Test:  0.9671

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         4
           1       0.67      1.00      0.80         2
           2       1.00      0.67      0.80         3
           3       1.00      1.00      1.00         2
           4       1.00      0.50      0.67         4
           5       1.00      1.00      1.00         1
           6       0.67      1.00      0.80         4
           7       0.83      1.00      0.91         5
           8       0.67      0.67      0.67         3
           9       1.00      1.00      1.00         4
       

Random Forest

In [ ]:
forest_model, forest_results = train_forest_model(train_data)

Fitting 5 folds for each of 20 candidates, totalling 100 fits

Best Random Forest parameters:
  n_estimators: 100
  min_samples_split: 5
  min_samples_leaf: 2
  max_features: None
  max_depth: 20
  class_weight: None
  bootstrap: True

Best cross-validation score: 0.9333

Training Time: 548.41 seconds

Accuracy:
  Train: 1.0000
  Test:  0.9556

Precision (weighted):
  Train: 1.0000
  Test:  0.9667

Recall (weighted):
  Train: 1.0000
  Test:  0.9556

F1-Score (weighted):
  Train: 1.0000
  Test:  0.9520

Matthews Correlation Coefficient:
  Train: 1.0000
  Test:  0.9535

Cohen's Kappa:
  Train: 1.0000
  Test:  0.9529

ROC AUC:
  Test:  0.9992

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       1.00      1.00      1.00         2
           2       1.00      0.80      0.89         5
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         5
   

Ensemble model

In [223]:
ensemble_model, ensemble_results = train_ensemble_model(train_data)

Fitting 3 folds for each of 10 candidates, totalling 30 fits

Training Time: 135.47 seconds

Accuracy:
  Train: 1.0000
  Test:  0.8667

Precision (weighted):
  Train: 1.0000
  Test:  0.8876

Recall (weighted):
  Train: 1.0000
  Test:  0.8667

F1-Score (weighted):
  Train: 1.0000
  Test:  0.8617

Matthews Correlation Coefficient:
  Train: 1.0000
  Test:  0.8614

Cohen's Kappa:
  Train: 1.0000
  Test:  0.8586

ROC AUC:
  Test:  0.9965

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.60      0.75         5
           1       0.50      0.50      0.50         2
           2       1.00      0.80      0.89         5
           3       1.00      1.00      1.00         3
           4       1.00      0.80      0.89         5
           5       1.00      1.00      1.00         2
           6       0.71      1.00      0.83         5
           7       1.00      0.86      0.92         7
           8       0.00      0.00      0.00         

XGBoosting model

In [224]:
boosting_model, boosting_results = train_boosting_model(train_data)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best XGBoost parameters: {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.1}

Training Time: 136.55 seconds

Accuracy:
  Train: 1.0000
  Test:  0.8556

Precision (weighted):
  Train: 1.0000
  Test:  0.8551

Recall (weighted):
  Train: 1.0000
  Test:  0.8556

F1-Score (weighted):
  Train: 1.0000
  Test:  0.8467

Matthews Correlation Coefficient:
  Train: 1.0000
  Test:  0.8486

Cohen's Kappa:
  Train: 1.0000
  Test:  0.8468

ROC AUC:
  Test:  0.9982

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.60      0.67         5
           1       0.50      0.50      0.50         2
           2       0.67      0.80      0.73         5
           3       1.00      1.00      1.00         3
           4       1.00      0.80      0.89         5
           5       0.67      1.00      0.80         2
           6       1.00      0.80      0.89         5
   

Stacking model

In [225]:
stacking_model, stacking_results = train_stacking_model(train_data)


STACKING MODEL: Multiple Base Learners + Meta-Learner

Training Time: 96.83 seconds

Accuracy:
  Train: 1.0000
  Test:  0.8889

Precision (weighted):
  Train: 1.0000
  Test:  0.9069

Recall (weighted):
  Train: 1.0000
  Test:  0.8889

F1-Score (weighted):
  Train: 1.0000
  Test:  0.8868

Matthews Correlation Coefficient:
  Train: 1.0000
  Test:  0.8837

Cohen's Kappa:
  Train: 1.0000
  Test:  0.8821

ROC AUC:
  Test:  0.9988

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.60      0.75         5
           1       0.50      0.50      0.50         2
           2       0.80      0.80      0.80         5
           3       1.00      1.00      1.00         3
           4       1.00      0.80      0.89         5
           5       1.00      1.00      1.00         2
           6       0.80      0.80      0.80         5
           7       1.00      1.00      1.00         7
           8       0.50      0.33      0.40         3
     

Neural Network

In [ ]:
nn_model, nn_results = train_neural_network(train_data)


Model Summary:


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_33 (Dense)                │ (None, 256)            │       132,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 20)             │           660 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 178,036 (695.45 KB)

 Trainable params: 177,140 (691.95 KB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - accuracy: 0.0333 - loss: 3.3791 - val_accuracy: 0.1190 - val_loss: 2.9086 - learning_rate: 0.0010
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.1013 - loss: 3.2552 - val_accuracy: 0.0952 - val_loss: 2.8661 - learning_rate: 0.0010
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.1356 - loss: 3.0061 - val_accuracy: 0.1429 - val_loss: 2.8258 - learning_rate: 0.0010
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.1051 - loss: 3.0526 - val_accuracy: 0.2143 - val_loss: 2.7890 - learning_rate: 0.0010
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.1096 - loss: 2.7919 - val_accuracy: 0.2381 - val_loss: 2.7525 - learning_rate: 0.0010
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.2080 - loss: 2.6483 - val_accuracy: 0.2857 - val_loss: 2.7090 - learning_rate: 0.0010
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.1834 - loss: 2.6762 - val_accuracy: 

ResMLP Model

In [ ]:
resmlp_model, resmlp_results = train_ResMLP(train_data)

[NN] Using class_weight: {0: 1.05, 1: 0.8076923076923077, 2: 1.05, 3: 0.875, 4: 1.05, 5: 0.8076923076923077, 6: 1.05, 7: 1.3125, 8: 0.875, 9: 1.05, 10: 1.1666666666666667, 11: 0.9545454545454546, 12: 0.9545454545454546, 13: 1.05, 14: 0.875, 15: 0.9545454545454546, 16: 1.5, 17: 0.8076923076923077, 18: 1.5, 19: 0.9545454545454546}
[NN] Using AdamW

Model Summary:


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 516)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_51 (Dense)    │ (None, 256)       │    132,352 │ input_layer_9[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dense_51[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_9        │ (None, 256)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_36          │ (None, 256)       │          0 │ activation_9[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_52 (Dense)    │ (None, 256)       │     65,792 │ dropout_36[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dense_52[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_10       │ (None, 256)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_37          │ (None, 256)       │          0 │ activation_10[0]… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_53 (Dense)    │ (None, 256)       │     65,792 │ dropout_37[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dense_53[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 256)       │          0 │ dropout_36[0][0], │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_11       │ (None, 256)       │          0 │ add_4[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_55 (Dense)    │ (None, 128)       │     32,896 │ activation_11[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_55[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_12       │ (None, 128)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_38          │ (None, 128)       │          0 │ activation_12[0]… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_56 (Dense)    │ (None, 128)       │     16,512 │ dropout_38[0][0]  │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 411,028 (1.57 MB)

 Trainable params: 408,212 (1.56 MB)

 Non-trainable params: 2,816 (11.00 KB)

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 17s 402ms/step - accuracy: 0.0522 - loss: 3.6975 - val_accuracy: 0.0476 - val_loss: 3.2862 - learning_rate: 3.0000e-04
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.0384 - loss: 3.5741 - val_accuracy: 0.0000e+00 - val_loss: 3.2677 - learning_rate: 3.0000e-04
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.0680 - loss: 3.3744 - val_accuracy: 0.0000e+00 - val_loss: 3.2539 - learning_rate: 3.0000e-04
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.0778 - loss: 3.4290 - val_accuracy: 0.0000e+00 - val_loss: 3.2486 - learning_rate: 3.0000e-04
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.0620 - loss: 3.2580 - val_accuracy: 0.0238 - val_loss: 3.2387 - learning_rate: 3.0000e-04
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.0670 - loss: 3.2208 - val_accuracy: 0.0238 - val_loss: 3.2278 - learning_rate: 3.0000e-04
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 

## Results <a name="Models-Results"></a>

In [243]:
# Model comparison
all_results = {
'baseline': baseline_results,
'forest': forest_results,
'ensemble': ensemble_results,
'boosting': boosting_results,
'stacking': stacking_results,
'neural_net': nn_results,
'lstm': resmlp_results
}
comparison_df = compare_all_models(all_results)


Model Performance Summary:
                      Model  Test Accuracy  Test F1  Test Precision  Test Recall  Test MCC  Test Kappa  Test AUC  Train Time (s)
               RandomForest       0.955556 0.951998        0.966667     0.955556  0.953479    0.952855  0.999228      548.407530
          Stacking_Ensemble       0.888889 0.886777        0.906896     0.888889  0.883698    0.882075  0.998841       96.833545
Baseline_LogisticRegression       0.866667 0.863038        0.893778     0.866667  0.860088    0.858464  0.967126        0.150504
             Ensemble_RF+GB       0.866667 0.861728        0.887566     0.866667  0.861389    0.858565  0.996490      135.471201
           Boosting_XGBoost       0.855556 0.846733        0.855093     0.855556  0.848578    0.846798  0.998174      136.554790
              NeuralNetwork       0.733333 0.723259        0.765802     0.733333  0.720262    0.717240  0.973815       17.211233
       NeuralNetwork+ResMLP       0.677778 0.668420        0.710811  

## PCA Comparison <a name="Model-PCA"></a>



In [230]:
pca_info = analyze_pca(train_data=train_data, feature_names=feature_names, n_components_range=(0.80, 0.90, 0.95, 0.99), random_state=42)

pca_configs = build_pca_configs(train_data=train_data, n_components_range=(0.80, 0.90, 0.95, 0.99), random_state=42)

pca_results = train_models_with_pca(train_data=train_data, pca_configs=pca_configs, variance_thresholds=(0.90, 0.95), random_state=42)



Variance Explained Analysis:
  80% variance: 22 components (4.3% of original features)
  90% variance: 36 components (7.0% of original features)
  95% variance: 50 components (9.7% of original features)
  99% variance: 83 components (16.1% of original features)

MODELS WITH PCA (90% variance) | components=36 | explained=0.9034

Training Time: 4.55 seconds

Accuracy:
  Train: 1.0000
  Test:  0.8333

Precision (weighted):
  Train: 1.0000
  Test:  0.8456

Recall (weighted):
  Train: 1.0000
  Test:  0.8333

F1-Score (weighted):
  Train: 1.0000
  Test:  0.8290

Matthews Correlation Coefficient:
  Train: 1.0000
  Test:  0.8243

Cohen's Kappa:
  Train: 1.0000
  Test:  0.8231

ROC AUC:
  Test:  0.9783

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.60      0.75         5
           1       0.50      0.50      0.50         2
           2       0.80      0.80      0.80         5
           3       1.00      1.00      1.00         3
 

In [250]:
# PCA analysis
pca_info = analyze_pca(train_data=train_data, feature_names=feature_names, n_components_range=(0.80, 0.90, 0.95, 0.99), random_state=42)

# Visualize variance
visualize_pca_analysis(pca_full=pca_info["pca_full"], cumsum_variance=pca_info["cumsum_variance"], thresholds=(0.80, 0.90, 0.95, 0.99), feature_names=feature_names, top_k=10, y_train=train_data["y_train"], max_samples=1000, random_state=42)

#  Visualize 2D/3D projection
pca3_model = visualize_pca_projection(X_train_scaled=train_data["X_train_scaled"], y_train=train_data["y_train"], random_state=42, max_samples=1000)



Variance Explained Analysis:
  80% variance: 22 components (4.3% of original features)
  90% variance: 36 components (7.0% of original features)
  95% variance: 51 components (9.9% of original features)
  99% variance: 85 components (16.5% of original features)


In [231]:
flat_rows = []
for pca_key, models in pca_results.items():
    for model_key, res in models.items():
        flat_rows.append({"PCA_Set": pca_key, "Model": res.get("model_name", model_key), "N_Components": res.get("n_components"), "Test_Accuracy": res.get("test_accuracy"), "Test_F1": res.get("test_f1"), "Test_MCC": res.get("test_mcc"), "Train_Time": res.get("train_time")})
pca_summary_df = pd.DataFrame(flat_rows).sort_values("Test_Accuracy", ascending=False)

print("\n PCA  summary:")
print(pca_summary_df.to_string(index=False))


PCA  summary:
PCA_Set              Model  N_Components  Test_Accuracy  Test_F1  Test_MCC  Train_Time
 pca_90 RandomForest_PCA90            36       0.866667 0.865532  0.859764    1.441524
 pca_95 RandomForest_PCA95            50       0.866667 0.869147  0.859990    1.485405
 pca_95  Baseline_LR_PCA95            50       0.855556 0.855468  0.847627    1.369620
 pca_90  Baseline_LR_PCA90            36       0.833333 0.829030  0.824257    4.547401
 pca_90    NeuralNet_PCA90            36       0.711111 0.699853  0.699292   23.930688
 pca_90      XGBoost_PCA90            36       0.700000 0.686335  0.684115    8.697115
 pca_95      XGBoost_PCA95            50       0.700000 0.707160  0.684846    6.166841
 pca_95    NeuralNet_PCA95            50       0.044444 0.021090 -0.008754    4.105562


In [244]:
n_original_features = train_data["X_train_scaled"].shape[1]
comparison_df = compare_pca_vs_original(original_results=all_results, pca_results=pca_results, n_original_features=n_original_features)


                      Model Feature_Set  N_Features  Test_Accuracy  Test_F1  Test_MCC  Train_Time
Baseline_LogisticRegression    Original         516       0.866667 0.863038  0.860088    0.150504
               RandomForest    Original         516       0.955556 0.951998  0.953479  548.407530
             Ensemble_RF+GB    Original         516       0.866667 0.861728  0.861389  135.471201
           Boosting_XGBoost    Original         516       0.855556 0.846733  0.848578  136.554790
          Stacking_Ensemble    Original         516       0.888889 0.886777  0.883698   96.833545
              NeuralNetwork    Original         516       0.733333 0.723259  0.720262   17.211233
       NeuralNetwork+ResMLP    Original         516       0.677778 0.668420  0.660400   39.516333
          Baseline_LR_PCA90     PCA_90%          36       0.833333 0.829030  0.824257    4.547401
         RandomForest_PCA90     PCA_90%          36       0.866667 0.865532  0.859764    1.441524
              XGBoos

In [245]:
# top-5 by accuracy
print("\nTop results by Test_Accuracy:")
print(comparison_df.sort_values("Test_Accuracy", ascending=False).head(5).to_string(index=False))


Top results by Test_Accuracy:
                      Model Feature_Set  N_Features  Test_Accuracy  Test_F1  Test_MCC  Train_Time
               RandomForest    Original         516       0.955556 0.951998  0.953479  548.407530
          Stacking_Ensemble    Original         516       0.888889 0.886777  0.883698   96.833545
Baseline_LogisticRegression    Original         516       0.866667 0.863038  0.860088    0.150504
             Ensemble_RF+GB    Original         516       0.866667 0.861728  0.861389  135.471201
         RandomForest_PCA90     PCA_90%          36       0.866667 0.865532  0.859764    1.441524


In [247]:
print_pca_summary_statistics(comparison_df=comparison_df, n_original_features=train_data["X_train_scaled"].shape[1])


Performance by Feature Set:
            Test_Accuracy                 Test_F1                 Train_Time  \
                     mean     std     max    mean     std     max       mean   
Feature_Set                                                                    
Original           0.8349  0.0957  0.9556  0.8289  0.0983  0.9520   139.1636   
PCA_90%            0.7778  0.0846  0.8667  0.7702  0.0904  0.8655     9.6542   
PCA_95%            0.6167  0.3890  0.8667  0.6132  0.4015  0.8691     3.2819   

                               
                  std     min  
Feature_Set                    
Original     188.5999  0.1505  
PCA_90%        9.9710  1.4415  
PCA_95%        2.3011  1.3696  
PCA BENEFITS ANALYSIS

PCA_90%:
  Feature reduction: 93.0%
  Average speedup: +1341.5%
  Average accuracy change: -6.84%
  Components used: 36

PCA_95%:
  Feature reduction: 90.3%
  Average speedup: +4140.4%
  Average accuracy change: -26.14%
  Components used: 50


##Models Different Features combinations <a name="Model-FS"></a>

In [263]:
rf_params = {"n_estimators": 100, "min_samples_split": 5, "min_samples_leaf": 2, "max_features": None, "max_depth": 20, "class_weight": None, "bootstrap": True, "n_jobs": -1, "random_state": 42}
fs_results = train_models_with_feature_selection(train_data=train_data, feature_names=feature_names, mi_df=mi_df, n_features_list=(50, 100, 200,300), rf_params=rf_params, random_state=42)


TRAINING WITH TOP 50 FEATURES 

Training Time: 0.19 seconds

Accuracy:
  Train: 0.9600
  Test:  0.8400

Precision (weighted):
  Train: 0.9649
  Test:  0.8664

Recall (weighted):
  Train: 0.9600
  Test:  0.8400

F1-Score (weighted):
  Train: 0.9572
  Test:  0.8421

Matthews Correlation Coefficient:
  Train: 0.9583
  Test:  0.8319

Cohen's Kappa:
  Train: 0.9578
  Test:  0.8306

ROC AUC:
  Test:  0.9829

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.50      0.57         4
           1       0.67      1.00      0.80         2
           2       1.00      0.67      0.80         3
           3       0.67      1.00      0.80         2
           4       0.67      0.50      0.57         4
           5       1.00      1.00      1.00         1
           6       1.00      0.75      0.86         4
           7       1.00      1.00      1.00         5
           8       0.50      0.67      0.57         3
           9       0.60      

In [264]:

fs_comparison_df = compare_feature_selection_results(original_results=all_results, feature_selection_results=fs_results, n_original_features=train_data["X_train_scaled"].shape[1])

# Visualisation
visualize_feature_selection_comparison(comparison_df=fs_comparison_df)



Comparison Table:
       Model  N_Features  Feature_Set  Test_Accuracy  Test_F1  Test_MCC  Train_Time
RandomForest         516 All Features       0.955556 0.951998  0.953479  548.407530
      LogReg          50       Top 50       0.840000 0.842089  0.831857    0.188468
RandomForest          50       Top 50       0.906667 0.905543  0.901813    1.407319
      LogReg         100      Top 100       0.866667 0.869309  0.860917    0.320896
RandomForest         100      Top 100       0.893333 0.885099  0.887840    2.511019
      LogReg         200      Top 200       0.893333 0.893172  0.888260    0.076882
RandomForest         200      Top 200       0.933333 0.917602  0.931078    3.972120
      LogReg         300      Top 300       0.880000 0.876718  0.874127    0.095980
RandomForest         300      Top 300       0.906667 0.891936  0.902846    6.546659

FEATURE SELECTION IMPACT ANALYSIS

RandomForest:
  Baseline (all features): Acc=0.9556, Time=548.41s
  Top 50: Acc=0.9067 (-4.89%), Time=1.4

# Best Model <a name="Best-Model"></a>

In [284]:

rf_best_params = {
    "n_estimators": 100,
    "min_samples_split": 5,
    "min_samples_leaf": 2,
    "max_features": None,
    "max_depth": 20,
    "class_weight": None,
    "bootstrap": True,
    "n_jobs": -1,
    "random_state": 42,
}

def train_best_model(train_data, rf_params, model_name="RandomForest_Best"):
    X_train = train_data["X_train_scaled"]
    y_train = train_data["y_train"]
    X_test  = train_data["X_test_scaled"]
    y_test  = train_data["y_test"]

    model = RandomForestClassifier(**rf_params)

    t0 = time.perf_counter()
    model.fit(X_train, y_train)
    fit_time = time.perf_counter() - t0

    t1 = time.perf_counter()
    y_pred_train = model.predict(X_train)
    y_pred_test  = model.predict(X_test)
    pred_time = time.perf_counter() - t1

    t2 = time.perf_counter()
    y_pred_proba = model.predict_proba(X_test)
    proba_time = time.perf_counter() - t2

    results = evaluate_model(
        model, model_name,
        {"y_train": y_train, "y_test": y_test},
        y_pred_train, y_pred_test, y_pred_proba,
        fit_time
    )

    results["fit_time_only"] = fit_time
    results["predict_time"] = pred_time
    results["predict_proba_time"] = proba_time

    print(f"\n[{model_name}] fit_time={fit_time:.4f}s | predict_time={pred_time:.4f}s | proba_time={proba_time:.4f}s")
    return model, results


In [274]:
#testing best model with params reduction

def train_best_model_200(train_data, feature_names, mi_df, n_features, rf_params, model_name=None):
    """
    Train RF on MI Top-N features and measure time (NO grid search).
    """
    if model_name is None:
        model_name = f"RandomForest_Best_Top{n_features}"

    top_features = mi_df.head(n_features)["Feature"].tolist()
    idx = [feature_names.index(f) for f in top_features]

    X_train = train_data["X_train_scaled"][:, idx]
    y_train = train_data["y_train"]
    X_test  = train_data["X_test_scaled"][:, idx]
    y_test  = train_data["y_test"]

    rf = RandomForestClassifier(**rf_params)

    t0 = time.perf_counter()
    rf.fit(X_train, y_train)
    fit_time = time.perf_counter() - t0

    t1 = time.perf_counter()
    y_pred_train = rf.predict(X_train)
    y_pred_test  = rf.predict(X_test)
    pred_time = time.perf_counter() - t1

    t2 = time.perf_counter()
    y_pred_proba = rf.predict_proba(X_test)
    proba_time = time.perf_counter() - t2

    results = evaluate_model(
        rf, model_name,
        {"y_train": y_train, "y_test": y_test},
        y_pred_train, y_pred_test, y_pred_proba,
        fit_time
    )
    results["fit_time_only"] = fit_time
    results["predict_time"] = pred_time
    results["predict_proba_time"] = proba_time
    results["n_features_used"] = n_features

    print(f"\n[{model_name}] fit_time={fit_time:.4f}s | predict_time={pred_time:.4f}s | proba_time={proba_time:.4f}s")
    return rf, results


In [285]:
rf_best_model, rf_best_results = train_best_model(train_data=train_data, rf_params=rf_best_params, model_name="RandomForest_Best")



Training Time: 8.04 seconds

Accuracy:
  Train: 1.0000
  Test:  0.9200

Precision (weighted):
  Train: 1.0000
  Test:  0.9071

Recall (weighted):
  Train: 1.0000
  Test:  0.9200

F1-Score (weighted):
  Train: 1.0000
  Test:  0.9049

Matthews Correlation Coefficient:
  Train: 1.0000
  Test:  0.9172

Cohen's Kappa:
  Train: 1.0000
  Test:  0.9149

ROC AUC:
  Test:  0.9985

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       1.00      1.00      1.00         2
           2       1.00      0.67      0.80         3
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         4
           5       1.00      1.00      1.00         1
           6       1.00      0.75      0.86         4
           7       0.71      1.00      0.83         5
           8       0.00      0.00      0.00         3
           9       1.00      1.00      1.00         4
       

In [286]:
rf_top200_model, rf_top200_results = train_best_model_200(train_data=train_data, feature_names=feature_names, mi_df=mi_df, n_features=200, rf_params=rf_best_params, model_name="RandomForest_Best_Top200")



Training Time: 6.06 seconds

Accuracy:
  Train: 1.0000
  Test:  0.9333

Precision (weighted):
  Train: 1.0000
  Test:  0.9150

Recall (weighted):
  Train: 1.0000
  Test:  0.9333

F1-Score (weighted):
  Train: 1.0000
  Test:  0.9176

Matthews Correlation Coefficient:
  Train: 1.0000
  Test:  0.9311

Cohen's Kappa:
  Train: 1.0000
  Test:  0.9291

ROC AUC:
  Test:  0.9978

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       1.00      1.00      1.00         2
           2       1.00      0.67      0.80         3
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         4
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         4
           7       0.83      1.00      0.91         5
           8       0.00      0.00      0.00         3
           9       1.00      1.00      1.00         4
       

In [287]:
rf_top200_results

{'model_name': 'RandomForest_Best_Top200',
 'train_time': 6.0550885249977,
 'train_accuracy': 1.0,
 'test_accuracy': 0.9333333333333333,
 'train_precision': 1.0,
 'test_precision': 0.915,
 'train_recall': 1.0,
 'test_recall': 0.9333333333333333,
 'train_f1': 1.0,
 'test_f1': 0.9176023976023976,
 'train_mcc': np.float64(1.0),
 'test_mcc': np.float64(0.9310784844652172),
 'train_kappa': np.float64(1.0),
 'test_kappa': np.float64(0.9291383219954649),
 'test_auc': np.float64(0.9977513227513228),
 'confusion_matrix': array([[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

We can clearly see that the Random Forest with cutting params to 200 is our best model.

# Final Prediction <a name="Conclusion"></a>

In [288]:
# Best model final

if "session_id" not in df_test.columns:
    cand = [c for c in df_test.columns if "session" in c.lower() or c.lower().startswith("ses")]
    if len(cand) == 0:
        raise ValueError(f"Не нашёл колонку session_id. Колонки: {list(df_test.columns)}")
    df_test = df_test.rename(columns={cand[0]: "session_id"})
    print(f"Renamed {cand[0]} -> session_id")

if "user_id" not in df_test.columns:
    df_test["user_id"] = "unknown"

df_test["session_id"] = df_test["session_id"].replace({"": np.nan, "nan": np.nan})
df_test = df_test.dropna(subset=["session_id"]).copy()

features_df_test, _ = engineer_features(df_test)

print("df_test:", df_test.shape, "| features_df_test:", features_df_test.shape)
features_df_test.head()



Features engineered: 516 features per session
Total sessions: 96
df_test: (819580, 13) | features_df_test: (96, 518)


,session_id,user_id,session_duration,event_count,avg_event_interval,std_event_interval,Gyroscope_count,Gyroscope_ratio,Accelerometer_count,Accelerometer_ratio,...,UserAcceleration_field7_mean,UserAcceleration_field7_std,UserAcceleration_field7_min,UserAcceleration_field7_max,UserAcceleration_field7_median,UserAcceleration_field7_q25,UserAcceleration_field7_q75,UserAcceleration_field7_iqr,UserAcceleration_field7_skew,UserAcceleration_field7_kurt
0,a47ea05f-68b6-4e84-8591-db9580d58ce6,unknown,166.051,6187,26.843033,719.811616,1018,0.164539,1018,0.164539,...,0,0,0,0,0,0,0,0,0,0
1,b582d088-0153-4f42-aa35-7a3f51c095b2,unknown,180.200,10739,16.781524,19.941116,1673,0.155787,2172,0.202253,...,0,0,0,0,0,0,0,0,0,0
2,a8654ddb-8aab-43e0-bef7-ae7ae88d2455,unknown,71.815,6893,10.420052,19.626463,668,0.096910,3503,0.508197,...,0,0,0,0,0,0,0,0,0,0
3,1f07e906-ccd6-4fa9-940c-6a96db17102d,unknown,114.836,11015,10.426366,19.014798,1067,0.096868,5599,0.508307,...,0,0,0,0,0,0,0,0,0,0
4,6d6e4f11-d787-4253-8070-aa352d202ab4,unknown,86.337,1897,45.536392,1335.317946,262,0.138113,569,0.299947,...,0,0,0,0,0,0,0,0,0,0


In [290]:
# test data pre_processing
if "scaler" in train_data:
    scaler = train_data["scaler"]

# привести набор колонок теста к feature_names (missing -> 0), порядок как в train
for col in feature_names:
    if col not in features_df_test.columns:
        features_df_test[col] = 0.0

X_val = features_df_test[feature_names].to_numpy()
X_val_scaled = scaler.transform(X_val)

session_ids_val = features_df_test["session_id"].to_numpy()

print("X_val_scaled:", X_val_scaled.shape)

X_val_scaled: (96, 516) | sessions: 96


In [291]:
# whole train df (train+test from train_data)
X_all = np.vstack([train_data["X_train_scaled"], train_data["X_test_scaled"]])
y_all = np.concatenate([train_data["y_train"], train_data["y_test"]])

# Feature Selection
TOP_N = 200
mi = mutual_info_classif(X_all, y_all, random_state=42)
mi_df_full = pd.DataFrame({"Feature": feature_names, "MI_Score": mi}).sort_values("MI_Score", ascending=False)
top_features = mi_df_full.head(TOP_N)["Feature"].tolist()
top_idx = [feature_names.index(f) for f in top_features]

X_all_top = X_all[:, top_idx]
X_val_top = X_val_scaled[:, top_idx]

# Best RF
rf_best_params = {
    "n_estimators": 100,
    "min_samples_split": 5,
    "min_samples_leaf": 2,
    "max_features": None,
    "max_depth": 20,
    "class_weight": None,
    "bootstrap": True,
    "n_jobs": -1,
    "random_state": 42
}

# Fit
final_model = RandomForestClassifier(**rf_best_params)
final_model.fit(X_all_top, y_all)


RandomForestClassifier(max_depth=20, max_features=None, min_samples_leaf=2,
                       min_samples_split=5, n_jobs=-1, random_state=42)

In [292]:
# predict + decode user_id
y_val_pred = final_model.predict(X_val_top)
user_pred = train_data["label_encoder"].inverse_transform(y_val_pred)

# final test df
submission_df = pd.DataFrame({"session_id": session_ids_val, "user_id": user_pred})
submission_df.to_csv("submission.csv", index=False)

print("submission:", submission_df.shape)
submission_df.head()

submission: (96, 2)


,session_id,user_id
0,a47ea05f-68b6-4e84-8591-db9580d58ce6,301eb109-b1fb-45b2-aae2-929c0886d820
1,b582d088-0153-4f42-aa35-7a3f51c095b2,301eb109-b1fb-45b2-aae2-929c0886d820
2,a8654ddb-8aab-43e0-bef7-ae7ae88d2455,301eb109-b1fb-45b2-aae2-929c0886d820
3,1f07e906-ccd6-4fa9-940c-6a96db17102d,301eb109-b1fb-45b2-aae2-929c0886d820
4,6d6e4f11-d787-4253-8070-aa352d202ab4,301eb109-b1fb-45b2-aae2-929c0886d820


In [294]:
len(submission_df.user_id.unique())

20

In [296]:
submission_df.to_csv('submission.csv', index = False)